<a href="https://colab.research.google.com/github/Giorgia3/Data_Spaces_Project/blob/main/esperimento_transformer_variations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer su dati covid

### Setup

In [ ]:
do_training = False
do_test = True
do_finetuning = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install -q tf-models-official==2.4.0

In [ ]:
from transformers import BertModel, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup,AutoTokenizer
from transformers import InputExample, InputFeatures, TrainingArguments, Trainer
import os.path
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, DBSCAN
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
import csv
import numpy as np
import numpy.ma as ma
import pandas as pd
from tqdm import tqdm
from enum import Enum
import tensorflow as tf
import itertools
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch import nn
import torch.nn.functional
import json
import tensorflow_hub as hub
import keras.backend as KB
from tensorflow.keras.callbacks import Callback
from official import nlp
import official.nlp.optimization
import time
import datetime
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TABLEAU_COLORS
from matplotlib.ticker import FuncFormatter, PercentFormatter
%matplotlib inline
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.axes_grid1.colorbar import colorbar
import matplotlib.cm as cm
from cycler import cycler
import seaborn as sns
import itertools
from linecache import getline
from pathlib import Path
import math
import gc
import json
import subprocess
import shlex
import pickle
from statsmodels.stats.proportion import proportion_confint, proportions_ztest
import collections
from scipy.stats import chisquare
import scipy.stats as stats
import plotly.express as px
import linecache
from os import write
from tabulate import tabulate
import networkx as nx

In [ ]:
!python --version
print(torch.__version__)

#### Configuration

In [ ]:
config_dict = {}
config_dict['USE_GPU'] = True
# USE_TPU = False
config_dict['TASK_TYPE'] = 'attention_analysis' # options:{'singularvalues_ratio_analysis', 'attention_flow', 'simple_test', 'distance_cones_analysis', 'attention_analysis', 'clustering', 'one_vs_all_classification', 'eigenvalues_analysis', 'singularvalues_ratio_analysis', 'von_neumann_entropy_attentions', 'distance_cones_analysis', 'shannon_entropy_attentions'}
config_dict['SPIKE_REGION_ANALYSIS'] = True

config_dict['REDUCE_N_INPUT_SAMPLES'] = True
config_dict['MAX_N_SAMPLES_PER_CLASS'] = 40000

config_dict['K'] = 12
config_dict['STRIDE'] = 9
config_dict['ADD_KMER_TOKENS_TO_VOCAB'] = True
config_dict['MIN_N_OCCUR_KMER'] = None

config_dict['ALIGNMENT'] = True
config_dict['SPLIT_DATA_IN_CHUNKS'] = False
if config_dict['SPLIT_DATA_IN_CHUNKS'] or config_dict['ALIGNMENT']:
    config_dict['CHUNK_LEN'] = 150
    config_dict['CHUNK_STRIDE'] = 150

config_dict['TRAIN_BATCH_SIZE'] = 4    # recommended: 32
config_dict['EVAL_BATCH_SIZE'] = 4    # recommended: 32
config_dict['EPOCHS'] = 2              # recommended: 2-4
config_dict['LR'] = 2e-5            #2e-5, #1e-3 # 2e-4

# maximum n. of tokens to be considered for each sequence: (CHUNK_LEN-K)/STRIDE +1 (max value supported by Bert-Base: 512)
if config_dict['SPLIT_DATA_IN_CHUNKS']:
    config_dict['MAX_LENGTH'] = math.ceil((config_dict['CHUNK_LEN']-config_dict['K'])/config_dict['STRIDE'] +1)  
else:
    config_dict['MAX_LENGTH'] = 512

config_dict['EMB_DIM'] = 768  

# attention threshold
config_dict['THETA'] = 0.3    

# MLP for OvsAC
config_dict['POSITIVE_CLASS_MLP'] = 'mu'   
config_dict['RELEVANT_TOKENS_MLP'] = ['211-214']   
config_dict['INPUT_DIM_MLP'] = len(config_dict['RELEVANT_TOKENS_MLP'])*768  
config_dict['OUTPUT_DIM_MLP'] = 2    
config_dict['TRAIN_BATCH_SIZE_MLP'] = 64   
config_dict['EVAL_BATCH_SIZE_MLP'] = 64    
config_dict['EPOCHS_MLP'] = 5              
config_dict['HIDDEN_UNITS_1_MLP'] = 32              # recommended: 2-4
config_dict['HIDDEN_UNITS_2_MLP'] = 32              # recommended: 2-4
config_dict['LR_MLP'] = 2e-5            #2e-5, #1e-3 # 2e-4

# WEA
all_layers_heads = []
for head in range(0,12):
    for layer in range(0,12):
        all_layers_heads.append([layer,head])
if config_dict['TASK_TYPE'] == 'distance_cones_analysis' :
    all_layers_heads1 = []
    for head in range(0,12):
        for layer in range(0,12):
            all_layers_heads1.append([layer,head])
    dc_layer_heads = all_layers_heads1
    # dc_layer_heads = []
    # for head in range(0,12):
    #     dc_layer_heads.append([1-1, head])
    config_dict['SELECTED_LAYER_HEAD_LIST'] = dc_layer_heads # distance cones:[[1-1,5-1]]
else:
    config_dict['SELECTED_LAYER_HEAD_LIST'] = all_layers_heads #[[1-1,4-1], [12-1,9-1], [11-1, 5-1], [1-1, 5-1], [5-1, 10-1]] #all_layers_heads 
config_dict['SELECTED_CLASS'] = "omicron"
config_dict['BASES'] = ['A','C','G','T','N', 'W', 'S', 'M', 'K', 'R', 'Y']

def generate_config_str():
    str_conf = '--------------------------------------------------------------\n'
    str_conf += "Configuration:\n"
    str_conf += "==============\n"
    str_conf += '\n'
    for key, value in config_dict.items():
        str_conf += f"\t{key} = {str(value)}\n"
    str_conf += '\n'
    str_conf += '--------------------------------------------------------------'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
    return str_conf

In [ ]:
colormaps = ['Greys', 'Greens', 'GnBu', 'Blues', 'Purples', 'PuRd', 'Reds', 'Oranges', 'YlOrBr', 
            'YlOrRd', 'OrRd', 'RdPu', 'BuPu',
            'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
colormaps_layers = ['Greys', 'Greens', 'YlGn', 'PuBuGn', 'GnBu', 'Blues',
             'Purples', 'RdPu', 'PuRd', 'Oranges', 'OrRd', 'Reds', 'YlOrRd', 'BuPu',
            'YlGnBu','BuGn', 'PuBu', 'YlOrBr']

In [ ]:
n_layers = 12
n_heads = 12

#### Directories and files

In [ ]:
main_dir = '/content/drive/My Drive/Dottorato/Bio/Transformer_variations'

# Input data:
datasets_dir = '/content/drive/My Drive/tesi/datasets'
variant_files = {}
for filename in os.listdir(Path(datasets_dir) / 'original'):
    variant_files[os.path.splitext(filename)[0]] = Path(datasets_dir) / 'original' / filename
if config_dict['SPIKE_REGION_ANALYSIS']:
    reference_seq_file = Path(datasets_dir) / 'reference.fasta'
    if not os.path.exists(reference_seq_file):
        raise FileNotFoundError(f'File {reference_seq_file} not found in {datasets_dir} folder.')

# Experiment dir:
if config_dict['SPLIT_DATA_IN_CHUNKS']:
    experiment_dir = Path(main_dir) / f"cl{config_dict['CHUNK_LEN']}_cs{config_dict['CHUNK_STRIDE']}_k{config_dict['K']}_s{config_dict['STRIDE']}"
else:
    experiment_dir = Path(main_dir) / f"k{config_dict['K']}_s{config_dict['STRIDE']}"

# Preprocessed data:
preprocessed_data_dir = Path(experiment_dir) / 'preprocessed_data'
if not os.path.exists(preprocessed_data_dir):
    os.makedirs(preprocessed_data_dir)
    print(f"Directory '{preprocessed_data_dir}' created")
if config_dict['SPIKE_REGION_ANALYSIS']:
    reformatted_seqs_file = Path(preprocessed_data_dir) / 'reformatted_seqs.fasta'
    duplicated_seqs_file = Path(preprocessed_data_dir) / 'duplicated_seqs.fasta'
    aligned_seqs_file = Path(preprocessed_data_dir) / 'aligned.sam'
    spike_seqs_file = Path(preprocessed_data_dir) / 'spike_seqs.csv'
    cigars_file = Path(preprocessed_data_dir) / 'cigars.csv'
    input_seqs_file = spike_seqs_file
else:
    reformatted_seqs_file = Path(preprocessed_data_dir) / 'reformatted_seqs.csv'
    input_seqs_file = reformatted_seqs_file
seqs_index_file = Path(preprocessed_data_dir) / 'seqs_index.csv'
ids_dict_file = Path(preprocessed_data_dir) / 'ids_dict.csv'
trainvaltest_splits_dir = Path(preprocessed_data_dir) / 'trainvaltest_splits'
train_file = Path(trainvaltest_splits_dir) / 'train.csv'
val_file = Path(trainvaltest_splits_dir) / 'val.csv'
test_file = Path(trainvaltest_splits_dir) / 'test.csv'
trainvaltest_sizes_file = Path(trainvaltest_splits_dir) / 'trainvaltest_sizes.csv'
token_count_file = Path(preprocessed_data_dir) / 'token_count.csv'

# Outputs dir:
outputs_dir = Path(experiment_dir) / 'outputs'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
    print(f"Directory '{outputs_dir}' created")
log_dir = Path(outputs_dir) / 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    print(f"Directory '{log_dir}' created")
tokens_histograms_dir = Path(outputs_dir) / 'tokens_histograms'
if not os.path.exists(tokens_histograms_dir):
    os.makedirs(tokens_histograms_dir)
    print(f"Directory '{tokens_histograms_dir}' created")
final_val_outputs_file = Path(outputs_dir) / 'final_val_outputs'
final_test_outputs_file = Path(outputs_dir) / 'final_test_outputs'
model_file_finetuned = Path(outputs_dir) / 'model_finetuned'
model_file = Path(outputs_dir) / 'model'
tokenizer_dir = Path(outputs_dir) / 'tokenizer'
training_stats_file = Path(outputs_dir) / 'training_stats'
test_accuracies_file = Path(outputs_dir) / 'test_accuracies'
final_data_test_file = Path(outputs_dir) / 'final_data_test_file'

# attention matrices
attention_matrices_dir = Path(outputs_dir) / 'attention_matrices'
if not os.path.exists(attention_matrices_dir):
    os.makedirs(attention_matrices_dir)
    print(f"Directory '{attention_matrices_dir}' created")
proportion_attn_domains_dir = Path(attention_matrices_dir) / "proportion_attn_domains"
if not os.path.exists(proportion_attn_domains_dir):
    os.makedirs(proportion_attn_domains_dir)
    print(f"Directory '{proportion_attn_domains_dir}' created")

#mathematical interpretation outputs
math_interpret_dir = Path(outputs_dir) / "mathematical_interpretation"
if not os.path.exists(math_interpret_dir):
    os.makedirs(math_interpret_dir)
    print(f"Directory '{math_interpret_dir}' created")
CLS_embeddings_dir = Path(math_interpret_dir) / "Y_outputs" / "CLS_embeddings"
if not os.path.exists(CLS_embeddings_dir):
    os.makedirs(CLS_embeddings_dir)
    print(f"Directory '{CLS_embeddings_dir}' created")

# biological interpretation output
log_dir_bio = Path(outputs_dir) / "biological_interpretation" / "log"
if not os.path.exists(log_dir_bio):
    os.makedirs(log_dir_bio)
    print(f"Directory '{log_dir_bio}' created")

# clustering output
clustering_dir = Path(outputs_dir) / "clustering"
if not os.path.exists(clustering_dir):
    os.makedirs(clustering_dir)
    print(f"Directory '{clustering_dir}' created")

In [ ]:
config_dict['CLASS_LABELS'] = {} 
i = 0
for variant_name in variant_files:
    config_dict['CLASS_LABELS'][variant_name] = i
    i += 1
config_dict['N_CLASSES'] = i
inv_class_labels_dict = {v:k for k, v in config_dict['CLASS_LABELS'].items()}

In [ ]:
n_log_files = len(os.listdir(log_dir))
if n_log_files > 0:
    log_file = Path(log_dir) / f'log({n_log_files}).txt'
else:
    log_file = Path(log_dir) / f'log.txt'

with open(log_file, 'w') as log_fp:
    log_fp.write(generate_config_str())
    print(generate_config_str())

#### Tokenizer and Bert model setup

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

model_base = BertForSequenceClassification.from_pretrained(
    "bert-base-cased", 
    num_labels=config_dict['N_CLASSES'],
    output_attentions = True, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
)
model_base.cuda()

model_finetuned = BertForSequenceClassification.from_pretrained(
    "bert-base-cased", 
    num_labels=config_dict['N_CLASSES'],
    output_attentions = True, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
)
model_finetuned.resize_token_embeddings(len(tokenizer))
model_finetuned.cuda()
model_finetuned.load_state_dict(torch.load(model_file_finetuned))

In [ ]:
if config_dict['USE_GPU'] and (do_training or do_test or config_dict['TASK_TYPE']=='one_vs_all_classification'):
    # Get the GPU device name.
    device_name = tf.test.gpu_device_name()

    # The device name should look like the following:
    if device_name == '/device:GPU:0':
        print('Found GPU at: {}'.format(device_name))
    else:
        raise SystemError('GPU device not found')


    # If there's a GPU available...
    if torch.cuda.is_available():    
        # Tell PyTorch to use the GPU.    
        device = torch.device("cuda")
        print('There are %d GPU(s) available.' % torch.cuda.device_count())
        print('We will use the GPU:', torch.cuda.get_device_name(0))
    # If not...
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")

# elif USE_TPU:
#     try:
#         tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#         print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
#     except ValueError:
#         raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

else:
    print('Runtime type: None.')

In [ ]:
# tokenizer.save_pretrained(tokenizer_file)

In [ ]:
# Tell pytorch to run this model on the GPU.
if do_training:
    model.cuda()

### Supervised: BERT classifier


####Training functions (fonti: [esempio1](https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb), [esempio2](https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX#scrollTo=6O_NbXFGMukX))

In [ ]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

def setup_training(train_data_size, log_fp):
    # Set up epochs and steps
    steps_per_epoch = int(train_data_size / config_dict['TRAIN_BATCH_SIZE'])
    num_train_steps = steps_per_epoch * config_dict['EPOCHS']
    warmup_steps = int(config_dict['EPOCHS'] * train_data_size * 0.1 / config_dict['TRAIN_BATCH_SIZE'])

    # creates an optimizer and learning rate scheduler
    optimizer = torch.optim.AdamW(model.parameters(),
                  lr = config_dict['LR'], # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = num_train_steps)
    # optimizer = nlp.optimization.create_optimizer(
    #     2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

    # Set the seed value all over the place to make this reproducible.
    seed_val = 42
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    log_fp.write("\nTraining setup:\n")
    log_fp.write("===============\n")
    log_fp.write(f"\tsteps_per_epoch = {steps_per_epoch}\n")
    log_fp.write(f"\tnum_train_steps = {num_train_steps}\n")
    log_fp.write(f"\twarmup_steps = {warmup_steps}\n")
    log_fp.write(f"\toptimizer = {optimizer}\n")
    log_fp.write(f"\tscheduler = {scheduler}\n")
    log_fp.write(f"\tseed_val = {seed_val}\n")
    
    return optimizer, scheduler

def loss_fn(outputs, targets):
    if config_dict['N_CLASSES'] > 2:
        criterion = torch.nn.CrossEntropyLoss()
        return criterion(outputs.logits, targets.to(torch.long))
    else:
        return outputs.loss

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Training and validation functions

In [ ]:
def train(train_dataloader, optimizer, scheduler, log_fp):
    # Perform one full pass over the training set.

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    train_steps_loss = []

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    for step, batch in enumerate(train_dataloader, 0):
        
        # Unpack this training batch from our dataloader. 
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)
        seq_ids = batch['seq_ids'].to(device, dtype = torch.int)
        y_onehot = torch.nn.functional.one_hot(targets.long(), num_classes=config_dict['N_CLASSES'])
        y_onehot = y_onehot.float()
            

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad() 

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        outputs = model(ids, 
                       token_type_ids=None, 
                       attention_mask=mask, 
                       labels=y_onehot,
                       return_dict=True)

        loss = loss_fn(outputs, targets)

        # Save hidden states of last layer for clustering
        # hidden_states = Tuple of torch.FloatTensor (one for the output of the embeddings
        # + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).
        # last_hidden_states = outputs.hidden_states[12]
        
        # Progress update every 40 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress.
            print('  Batch {:>5}  of  {:>5}.    Elapsed: {:}.     Loss:  {:}.'.format(step, len(train_dataloader), elapsed, loss.item()))
            log_fp.write(('  Batch {:>5}  of  {:>5}.    Elapsed: {:}.     Loss:  {:}.\n'.format(step, len(train_dataloader), elapsed, loss.item())))
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += float(loss.item())
        if step % 50 == 0 and not step == 0:
            train_steps_loss.append(float(loss.item()))
        
        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    log_fp.write("  Average training loss: {0:.2f}\n".format(avg_train_loss))
    log_fp.write("  Training epoch took: {:}\n".format(training_time))

    return avg_train_loss, training_time, train_steps_loss


def validation(val_dataloader, log_fp):
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    final_data = {'seq_ids' : [],
                  'positions' : [],
                  'targets' : [],
                  'outputs' : [],
                  'last_hidden_states_dict' : {}
                  }

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Unpack this training batch from our dataloader. 
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)
        seq_ids = batch['seq_ids'].to('cpu').numpy()
        positions = batch['positions'].to('cpu').numpy()

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            y_onehot = torch.nn.functional.one_hot(targets.long(), num_classes=config_dict['N_CLASSES'])
            y_onehot = y_onehot.float()
            outputs = model(ids,  
                            attention_mask=mask,
                            labels=y_onehot,
                            return_dict=True)
            
        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = loss_fn(outputs, targets)
        logits = outputs.logits
            
        # Accumulate the validation loss.
        total_eval_loss += float(loss.item())

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = targets.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
        final_data['targets'].extend(y_onehot.cpu().detach().numpy().tolist())
        if config_dict['N_CLASSES'] > 2:
            final_data['outputs'].extend(torch.softmax(outputs.logits, dim=1).cpu().detach().numpy().tolist())
        else:
            final_data['outputs'].extend(torch.sigmoid(outputs.logits).cpu().detach().numpy().tolist())
        final_data['seq_ids'].extend(seq_ids)
        final_data['positions'].extend(positions)

        # Save hidden states of last layer for clustering
        # hidden_states = Tuple of torch.FloatTensor (one for the output of the embeddings
        # + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).
        last_hidden_states = outputs.hidden_states[12].cpu().detach().numpy()
        for i, seq_id in enumerate(seq_ids):
            final_data['last_hidden_states_dict']['seq_id'] = last_hidden_states[i]


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Validation Accuracy: {0:.2f}".format(avg_val_accuracy))
    log_fp.write("  Validation Accuracy: {0:.2f}\n".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    log_fp.write("  Validation Loss: {0:.2f}\n".format(avg_val_loss))
    log_fp.write("  Validation took: {:}\n".format(validation_time))

    return final_data, avg_val_loss, avg_val_accuracy, validation_time


def supervisedBertClassifierFinetune(train_dataloader, val_dataloader, train_data_size, log_fp):
    gc.collect()
    torch.cuda.empty_cache()
    optimizer, scheduler = setup_training(train_data_size, log_fp)

    training_stats = []
    train_steps_loss = []
    total_t0 = time.time()
    log_fp.write("\nTraining and validation:\n")
    log_fp.write("==========================\n")

    for epoch in range(config_dict['EPOCHS']):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch + 1, config_dict['EPOCHS']))
        print('Training...')
        log_fp.write('\n======== Epoch {:} / {:} ========\n'.format(epoch + 1, config_dict['EPOCHS']))
        log_fp.write("\nTraining:\n")
        avg_train_loss, training_time, train_steps_loss_epoch = train(train_dataloader, optimizer, scheduler, log_fp)
        train_steps_loss.extend(train_steps_loss_epoch)

        print("")
        print("Validation...")
        log_fp.write("\nValidation:\n")
        final_data, avg_val_loss, avg_val_accuracy, validation_time = validation(val_dataloader, log_fp)

        training_stats_epoch = {
            'epoch': epoch + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time,
            'final_data':final_data
        }
        training_stats.append(training_stats_epoch)


    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    log_fp.write("Total training took {:} (h:mm:ss)\n".format(format_time(time.time()-total_t0)))

    # return train hidden_states from last epoch
    return training_stats, train_steps_loss

#### Finetuning



In [ ]:
if do_training and do_finetuning:
    with open(train_file) as train_fp, open(val_file) as val_fp, open(log_file, 'a') as log_fp:

        train_reader = csv.reader(train_fp, delimiter=',')
        train_metadata = {'len':sizes_info['train_data_size_seqs']}
        X_train_generator = DatasetGenerator(train_reader, train_fp, train_metadata) 
        train_dataloader = DataLoader(
                X_train_generator,  # The training samples.
                sampler = RandomSampler(X_train_generator), # Select batches randomly
                batch_size = config_dict['TRAIN_BATCH_SIZE'], # Trains with this batch size.
                num_workers = 0
        )
        
        val_reader = csv.reader(val_fp, delimiter=',')
        val_metadata = {'len':sizes_info['val_data_size_seqs']}
        X_val_generator = DatasetGenerator(val_reader, val_fp, val_metadata) #.batch(EVAL_BATCH_SIZE)

        
        validation_dataloader = DataLoader(
                X_val_generator, # The validation samples.
                sampler = SequentialSampler(X_val_generator), # Pull out batches sequentially.
                batch_size = config_dict['EVAL_BATCH_SIZE'], # Evaluate with this batch size.
                num_workers = 0
        )

        training_stats, train_steps_loss = supervisedBertClassifierFinetune(train_dataloader, validation_dataloader, sizes_info['train_data_size_seqs'], log_fp)

#### Save model and outputs

In [ ]:
# save or load model for inference:
if do_training or do_test or config_dict['TASK_TYPE']=='one_vs_all_classification':
    if do_finetuning:
        mod_f = model_file_finetuned
    else:
        mod_f = model_file
    if os.path.exists(mod_f):
        # model.load_state_dict(torch.load(mod_f, map_location=torch.device('cpu')))
        # device = torch.device("cpu")
        model_finetuned.load_state_dict(torch.load(mod_f))
    else:
        torch.save(model_finetuned.state_dict(), mod_f)

if do_test or config_dict['TASK_TYPE']=='one_vs_all_classification':
    model_finetuned.cuda()

In [ ]:
# # save or load last epoch validation data:
# if os.path.exists(final_val_outputs_file):
#     final_data = pickle.load(open(final_val_outputs_file, 'rb'))
# else:         
#     final_data = training_stats[-1]['final_data']
#     # save output data on file for furure computation:
#     pickle.dump(final_data, open(final_val_outputs_file, 'wb'))
#     # pickle.dump(str(final_data), open(final_val_outputs_file, 'wb'))


# # save or load last epoch validation data:
# if os.path.exists(training_stats_file):
#     training_stats = pickle.load(open(training_stats_file, 'rb'))
# else:         
#     # save output data on file for furure computation:
#     pickle.dump(training_stats, open(training_stats_file, 'wb'))
#     # pickle.dump(str(final_data), open(final_val_outputs_file, 'wb'))

#### Validation statistics

Cluster last hidden states

In [ ]:
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt # graphs plotting
# import matplotlib.cm as cm
# import seaborn as sns

# print("Starting t-SNE")

# X_embedded = TSNE(n_components = 2, perplexity = 30, random_state = 1).fit_transform()

# print("Writting File!!!")

# write_path_11 = "/alina-data1/sarwan/IEEE_BigData/Dataset/t_sne_plot_2_dim.csv"

Statistics and plots functions

In [ ]:
from pandas._config import config
def show_train_stats_and_plots(training_stats, train_steps_loss):
    print('Results:')
    # Display floats with two decimal places.
    pd.set_option('precision', 2)

    # Create a DataFrame from our training statistics.
    df_stats = pd.DataFrame(data=training_stats)

    # Use the 'epoch' as the row index.
    df_stats = df_stats.set_index('epoch')

    # A hack to force the column headers to wrap.
    #df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

    # Display the table.
    print(df_stats)

    # Use plot styling from seaborn.
    sns.set(style='darkgrid')

    # Increase the plot size and font size.
    sns.set(font_scale=1.5)
    fig = plt.figure(1, figsize=(12,6))
    #plt.rcParams["figure.figsize"] = (12,6)

    # Plot the learning curve.
    plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
    plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

    # Label the plot.
    plt.title("Average Training & Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.xticks([1, 2, 3, 4])

    plt.show() 
    fig_path = Path(outputs_dir) / 'trainval.jpg'
    fig.savefig(fig_path)  

    fig2 = plt.figure(2, figsize=(30,6))
    plt.plot(train_steps_loss, 'b', label="Training steps loss (every 40 steps)")

    # Label the plot.
    plt.title("Training steps loss")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.legend()

    plt.show() 
    fig_path2 = Path(outputs_dir) / 'train_steps.jpg'
    fig2.savefig(fig_path2)  


def confusion_matrix_plot(targets_labels, outputs_labels, path, taskname = ""):
    confusion_matr = metrics.confusion_matrix(targets_labels, outputs_labels)
    plt.figure()
    plt.figure(figsize=(15, 15))
    sns.heatmap(confusion_matr, annot=True, fmt="d", cmap="viridis", annot_kws={"fontsize":20})
    plt.title(f'{taskname} Confusion Matrix', fontsize=25)
    plt.ylabel('True label', fontsize=21)
    plt.xlabel('Predicted label', fontsize=21)
    curr_xticks, curr_xlabels = plt.xticks()
    plt.xticks(curr_xticks, labels=[inv_class_labels_dict[int(t.get_text())] for t in curr_xlabels], rotation=90, fontsize=20)
    curr_yticks, curr_ylabels = plt.yticks()
    plt.yticks(curr_yticks, labels=[inv_class_labels_dict[int(t.get_text())] for t in curr_ylabels], rotation=0, fontsize=20)
    cax = plt.gcf().axes[-1]
    cax.tick_params(labelsize=20)
    fig_path = Path(path) / f"{taskname.replace(' ', '_')}_confusion_matrix.jpg"
    fig = plt.gcf()
    fig.savefig(fig_path)
    plt.show()
    plt.figure()


def plot_PRC(y, y_score, y_pred, path):
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim([-0.1, 1.1])
    ax.set_ylim([-0.1, 1.1])
    ax.grid(True)
    ax.set_aspect('equal')
    ax.legend(loc='lower right')
    ax.set_title(f"Precision-Recall Curves")
    # metrics.PrecisionRecallDisplay.from_predictions(y, y_score, ax=ax, )
    # precision = dict()
    # recall = dict()
    # average_precision = dict()
    # for i in range(config_dict['N_CLASSES']):
    #     precision[i], recall[i], _ = metrics.precision_recall_curve(y[:],
    #                                                         y_score[:, i])
    #     average_precision[i] = metrics.average_precision_score(y[:], y_score[:, i])

    # A "micro-average": quantifying score on all classes jointly
    # precision["micro"], recall["micro"], _ = metrics.precision_recall_curve(y,
    #     y_score)
    # average_precision["micro"] = metrics.average_precision_score(y, y_score,
    #                                                     average="micro")
    # print('Average precision score, micro-averaged over all classes: {0:0.2f}'
    #     .format(average_precision["micro"]))
    # display = metrics.PrecisionRecallDisplay(
    #     recall=recall["micro"],
    #     precision=precision["micro"],
    #     average_precision=average_precision["micro"])
    # display.plot()
    # _ = display.ax_.set_title(f"Precision-Recall curve micro-averaged over all classes (AP={average_precision['micro']})")
    def multiclass_prc(y_test, y_score, y_pred, average="micro"):
        lb = LabelBinarizer()
        lb.fit(y_test)
        y_test = lb.transform(y_test)
        y_pred = lb.transform(y_pred)

        for (idx, c_label) in enumerate(config_dict['CLASS_LABELS'].keys()):
            display = metrics.PrecisionRecallDisplay.from_predictions(y_test[:,idx].astype(int), y_pred[:,idx], name=c_label)
            display.plot(ax=ax)
        # ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
        return metrics.average_precision_score(y_test, y_pred, average=average)
    print('AP score:', multiclass_prc(y, y_score, y_pred))
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    # fig.show()
    fig.savefig(Path(path) / 'prc.png')

def ROC_curve_plot(targets_labels, outputs_labels, path, taskname = "", figsize=(17, 6)):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    labels_dict = {class_label:[[],[]] for class_label in config_dict['CLASS_LABELS'].values()}
    for i in range(len(targets_labels)):
        labels_dict[targets_labels[i]][0].append(targets_labels[i])
        labels_dict[targets_labels[i]][1].append(outputs_labels[i])

    for class_label, tgt_out_labels in labels_dict.items():
        print("CLASSSSS")
        print(class_label)
        print(tgt_out_labels[0])
        print(tgt_out_labels[1])
        fpr[class_label], tpr[class_label], _ = metrics.roc_curve(tgt_out_labels[0], tgt_out_labels[1], pos_label=class_label)
        roc_auc[class_label] = metrics.auc(fpr[class_label], tpr[class_label])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for class_label in labels_dict.keys():
        ax.plot(fpr[class_label], tpr[class_label], label='ROC curve (AUC = %0.2f) for class %i' % (roc_auc[class_label], class_label))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

def final_statistics(target_labels, output_labels, output_logits, log_file, taskname, logits=None, target_names=None):
    accuracy = metrics.accuracy_score(target_labels, output_labels)
    f1_score_micro = metrics.f1_score(target_labels, output_labels, average='micro')
    f1_score_macro = metrics.f1_score(target_labels, output_labels, average='macro')
    if target_names==None:
        target_names = config_dict['CLASS_LABELS']
    specificity = {}
    for l_name, l in target_names.items():
        prec,recall,_,_ = metrics.precision_recall_fscore_support(target_labels==l,
                                                        output_labels==l,
                                                        pos_label=True,average=None)
        specificity[l_name] = recall[0] 
    classification_report = metrics.classification_report(target_labels, output_labels, target_names=target_names.keys())
    confusion_matr = metrics.confusion_matrix(target_labels, output_labels)
    print(f'{taskname}:')
    print(f"===============================")
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    print(f"Specificity:\n{json.dumps(specificity, indent=4)}\n")
    print(f"Classification report:\n{str(classification_report)}")
    confusion_matrix_plot(target_labels, output_labels, outputs_dir, taskname)
    # plot_PRC(target_labels, output_logits, output_labels, outputs_dir)
    with open(log_file, 'a') as log_fp:
        log_fp.write(f"\n{taskname}:\n")
        log_fp.write(f"===============================\n")
        log_fp.write(f"Accuracy Score = {accuracy}\n")
        log_fp.write(f"F1 Score (Micro) = {f1_score_micro}\n")
        log_fp.write(f"F1 Score (Macro) = {f1_score_macro}\n")
        log_fp.write(f"Classification report:\n{classification_report}\n")
        log_fp.write(f"Specificity:\n{json.dumps(specificity, indent=4)}\n")
        log_fp.write(f"Confusion matrix:\n{confusion_matr}\n")
    if logits:
        logits_pos_score = [x[target_labels[i]] for i,x in enumerate(logits)]
        ROC_curve_plot(target_labels, logits_pos_score, outputs_dir, taskname)

def find_best_positions(final_data, min_score=0.8, threshold=0.5):
    best_positions = None

    outputs_labels=np.argmax(final_data['outputs'],axis=1)
    targets_labels=np.argmax(final_data['targets'],axis=1)

    # analisys according to position in sequence
    final_data['select_pred'] = []
    final_data['count_pred'] = []
    for target,pred,pred_score in zip(targets_labels,outputs_labels, final_data['outputs']):
        # select only predictions which are correct and with score > min_score (*)
        final_data['select_pred'].append(int(target == pred and pred_score[np.argmax(pred_score)]>min_score)) 
        final_data['count_pred'].append(1)
    final_data_df = pd.DataFrame(final_data).drop(columns=['outputs', 'targets','seq_ids'])
    grouped_df = final_data_df.groupby(['positions']).agg({'select_pred': 'sum', 'count_pred': 'count'})

    # for each position, calculate percentage of chunks which satisfy conditions (see (*))
    grouped_df['percents'] = grouped_df['select_pred'] / grouped_df['count_pred']
    # select only positions which have a percentage of chunks which satisfy conditions (see (*)) > threshold
    best_positions = [i for i, perc in enumerate(np.asarray(grouped_df['percents'])) if perc > threshold]

    print(f"Best positions: {best_positions}")
    with open(log_file, 'a') as log_fp:
        log_fp.write(f"\nBest positions (i.e. those with >{threshold*100}% of chunks with correct prediction and score>{min_score}):\n")
        log_fp.write(f"=======================================================================================\n")
        log_fp.write(f"{best_positions}\n")

    plot = grouped_df['percents'].plot.bar(figsize=(20,10))
    fig = plot.get_figure()
    fig.suptitle(f"For each position, percentage of chunks with correct prediction and score>{min_score}", y=0.95, fontsize=20)
    plt.yticks(np.arange(0, 1, 0.1))
    plt.grid()
    plt.axhline(y=threshold, color='r', linestyle='-')
    fig_path = Path(outputs_dir) / f"percent_pos.jpg"
    fig.savefig(fig_path)
    plt.show()

    return best_positions

def per_sample_result_computation(final_data, best_positions, min_score=0.8, taskname='', filter_positions=True, filter_score=True):
    targets_dict = {}
    preds_dict = {}
    for id,target in zip(final_data['seq_ids'],final_data['targets']):
        if id not in targets_dict:
            targets_dict[id] = np.argmax(target)
        if id not in preds_dict:
            preds_dict[id] = {'counts':[], 'outputs_label':[], 'prediction': []}


    final_data['select_pred'] = []
    final_data['outputs_label'] = []
    for pos,pred in zip(final_data['positions'],final_data['outputs']):
        # select prediction only if it is in best position and if prediction is certain (score>min_score) (*)
        if filter_positions:
            if filter_score: 
                final_data['select_pred'].append(int(pos in best_positions and pred[np.argmax(pred)]>min_score))
            else:
                final_data['select_pred'].append(int(pos in best_positions))
        else:
            if filter_score: 
                final_data['select_pred'].append(int(pred[np.argmax(pred)]>min_score))
            else:
                final_data['select_pred'].append(int(pred[np.argmax(pred)]>0))
        final_data['outputs_label'].append(np.argmax(pred))
    
    final_data_df = pd.DataFrame(final_data)
    # filter only selected predictions according to conditions (see (*))
    filtered_data_df = final_data_df[final_data_df['select_pred'] > 0]
    filtered_data_df = filtered_data_df[['seq_ids','outputs_label']]
    grouped_sample_data_df = pd.DataFrame(filtered_data_df).groupby(['seq_ids','outputs_label']).size().reset_index(name='counts')
    grouped_sample_data_dict = grouped_sample_data_df.to_dict('list')

    for seq_id,output_l,count in zip(grouped_sample_data_dict['seq_ids'], grouped_sample_data_dict['outputs_label'], grouped_sample_data_dict['counts']):
        preds_dict[seq_id]['outputs_label'].append(output_l)
        preds_dict[seq_id]['counts'].append(count)

     
    for seq_id in preds_dict.keys():
        counts_sorted = preds_dict[seq_id]['counts'].copy()
        counts_sorted.sort(reverse=True)
        if len(preds_dict[seq_id]['counts'])==0 or (len(counts_sorted)>1 and all(element == counts_sorted[0] for element in counts_sorted)):
            preds_dict[seq_id]['prediction'] = 'uncertain'
        else:
            majority_class_index = np.argmax(preds_dict[seq_id]['counts'])
            preds_dict[seq_id]['prediction'] = preds_dict[seq_id]['outputs_label'][majority_class_index]

    correct_pred_count = 0
    uncertain_pred_count = 0
    tot_count = 0
    target_labels = []
    output_labels = []


    with open(log_file, 'a') as log_fp:
        for seq_id in targets_dict.keys():
            tot_count += 1
            if preds_dict[seq_id]['prediction'] == 'uncertain':
                uncertain_pred_count += 1
            else:
                target_labels.append(targets_dict[seq_id])
                output_labels.append(preds_dict[seq_id]['prediction'])
                if targets_dict[seq_id] == preds_dict[seq_id]['prediction']:
                    correct_pred_count += 1
            print(f"seq: {seq_id}\t target: {targets_dict[seq_id]}\t predicted: {preds_dict[seq_id]['prediction']}")
            log_fp.write(f"seq: {seq_id}\t target: {targets_dict[seq_id]}\t predicted: {preds_dict[seq_id]['prediction']}\n")

    final_statistics(target_labels, output_labels, log_file, f"{taskname} grouped by samples, filter_positions={filter_positions}, filter_score={filter_score}, min_score={min_score}")
    
    print(f"Correct predictions: {correct_pred_count}/{tot_count} -> {correct_pred_count/tot_count}")
    print(f"Wrong predictions: {tot_count-correct_pred_count-uncertain_pred_count}/{tot_count} -> {(tot_count-correct_pred_count-uncertain_pred_count)/tot_count}")
    print(f"Uncertain predictions: {uncertain_pred_count}/{tot_count} -> {uncertain_pred_count/tot_count}")

    with open(log_file, 'a') as log_fp:
        log_fp.write(f"Correct predictions: {correct_pred_count}/{tot_count} -> {correct_pred_count/tot_count}\n")
        log_fp.write(f"Wrong predictions: {tot_count-correct_pred_count-uncertain_pred_count}/{tot_count} -> {(tot_count-correct_pred_count-uncertain_pred_count)/tot_count}\n")
        log_fp.write(f"Uncertain predictions: {uncertain_pred_count}/{tot_count} -> {uncertain_pred_count/tot_count}\n")



In [ ]:
if do_training:
    show_train_stats_and_plots(training_stats, train_steps_loss)

    output_labels=np.argmax(final_data['outputs'],axis=1)
    target_labels=np.argmax(final_data['targets'],axis=1)
    final_statistics(target_labels, output_labels, final_data['outputs'], log_file, 'Global Validation')

    if config_dict['SPLIT_DATA_IN_CHUNKS']:
        #best_positions = find_best_positions(final_data, min_score=0.8, threshold=0.75)
        #per_sample_result_computation(final_data, best_positions, taskname="Validation")
        per_sample_result_computation(final_data, best_positions, taskname="Validation", filter_positions=True, filter_score=True, min_score=0.8)

In [ ]:
# show_train_stats_and_plots(training_stats, train_steps_loss)

# output_labels=np.argmax(final_data['outputs'],axis=1)
# target_labels=np.argmax(final_data['targets'],axis=1)
# output_logits = final_data['outputs']
# final_statistics(target_labels, output_labels, output_logits, log_file, 'Validation')

####Attention matrices functions

In [ ]:
def get_attentions(attentions, sample_idx_in_batch, layer=0, attention_head=0, sum=False):
  '''
  get the particular output for a particular layer and attention head
  layer -> 0 to 11
  attention_head -> 0 to 11
  '''
  if sum:
    #avg over all attention heads in a layer
    return attentions[layer][sample_idx_in_batch].sum(dim=0).cpu().detach().numpy()

  #return values for a particular attention head inside a specific layer
  return attentions[layer][sample_idx_in_batch][attention_head].cpu().detach().numpy()


def min_max_scale(X, range=(0, 1)):
    mi, ma = range
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (ma - mi) + mi
    return X_scaled


# def calculate_weight_domain_proportion_by_head(attentions_sum, domain, proportion, theta=0):
#     if domain not in domain_coordinates_1based:
#         raise ValueError("plt_attentions_domain: domain must be one of %r." % domain_coordinates_1based)

#     domain_start_idx, domain_end_idx = convert_domain_coords_in_token_indices(domain)

#     # proportion of attention weights in the selected domain from each head that connects all amino acids to those in the selected domain
#     #weight_total_by_head = torch.zeros((n_layers, n_heads), dtype=torch.double)
#     if proportion == "attention":
#         # Update total attention_analysis weights per head. Sum over from_index (dim 2), to_index (dim 3)
#         weight_total_by_head = attentions_sum.sum((2, 3))
#         # Update weighted sum of feature values per head
#         weight_domain_sum_by_head = attentions_sum[:, :, :, domain_start_idx:domain_end_idx+1].sum((2, 3))
#     elif proportion == "high-confidence-attention":
#         attentions_sum_total_masked = ma.masked_where(attentions_sum <= theta, attentions_sum)
#         ma.set_fill_value(attentions_sum_total_masked, 0)
#         weight_total_by_head = attentions_sum_total_masked.sum((2, 3))
#         attentions_sum_domain = attentions_sum[:, :, :, domain_start_idx:domain_end_idx+1]
#         attentions_sum_domain_masked = ma.masked_where(attentions_sum_domain <= theta, attentions_sum_domain)
#         ma.set_fill_value(attentions_sum_domain_masked, 0)
#         weight_domain_sum_by_head = attentions_sum_domain_masked.sum((2, 3))
#     elif proportion == "high-attention-tokens-count":
#         mask_high_attn_total = attentions_sum > theta
#         weight_total_by_head = mask_high_attn_total.long().sum((2, 3))
#         mask_high_attn_domain = attentions_sum[:, :, :, domain_start_idx:domain_end_idx+1] > theta
#         weight_domain_sum_by_head = mask_high_attn_domain.long().sum((2, 3))
#     else:
#         raise ValueError("plt_attentions_domain: proportion must be one of %r." % ["attention", "high-attention-tokens"])

#     return weight_domain_sum_by_head / weight_total_by_head, weight_domain_sum_by_head, weight_total_by_head


# def plt_attentions_domain(attentions_sum, domain, dir_path, filename, proportion, theta=0, min_total=0, fig_size=(130,100), title=None):
#     if domain not in domain_coordinates_1based:
#         raise ValueError("plt_attentions_domain: domain must be one of %r." % domain_coordinates_1based)

#     weight_domain_proportion_by_head, weight_sum, weight_total = calculate_weight_domain_proportion_by_head(attentions_sum, domain, proportion, theta=theta)

#     exclude_mask = np.array(weight_total) < min_total
#     masked_weight_domain_proportion = np.ma.masked_array(weight_domain_proportion_by_head, mask=exclude_mask)
#     layer_max = np.asarray(masked_weight_domain_proportion).max(-1)

#     fig = plt.figure(figsize=(5, 3.5)) #plt.figure(figsize=(3, 2.2))
#     fig.suptitle(title, fontsize=13, fontweight='bold')
#     ax1 = plt.subplot2grid((100, 85), (0, 0), colspan=65, rowspan=99)  # Heatmap
#     ax2 = plt.subplot2grid((100, 85), (12, 70), colspan=15, rowspan=75)  # Barchart

#     xtick_labels = [str(i) if i % 2 == 0 else '' for i in range(1, n_heads + 1)]
#     ytick_labels = [str(i) if i % 2 == 0 else '' for i in range(1, n_layers + 1)]
#     heatmap = sns.heatmap((masked_weight_domain_proportion * 100).tolist(), center=0.0, ax=ax1,
#                           square=True, cbar=False, linewidth=0.1, linecolor='#D0D0D0',
#                           cmap=LinearSegmentedColormap.from_list('rg', ["#F14100", "white", "#5a3dc4"], N=256),
#                           mask=exclude_mask,
#                           xticklabels=xtick_labels,
#                           yticklabels=ytick_labels)
#     for _, spine in heatmap.spines.items():
#         spine.set_visible(True)
#         spine.set_edgecolor('#D0D0D0')
#         spine.set_linewidth(0.1)
#     plt.setp(heatmap.get_yticklabels(), fontsize=11)
#     plt.setp(heatmap.get_xticklabels(), fontsize=11)
#     heatmap.tick_params(axis='x', pad=1, length=2)
#     heatmap.tick_params(axis='y', pad=.5, length=2)
#     heatmap.yaxis.labelpad = 3
#     heatmap.invert_yaxis()
#     heatmap.set_facecolor('#E7E6E6')
#     # split axes of heatmap to put colorbar
#     ax_divider = make_axes_locatable(ax1)
#     cax = ax_divider.append_axes('left', size='7%', pad='33%')
#     # # make colorbar for heatmap.
#     # # Heatmap returns an axes obj but you need to get a mappable obj (get_children)
#     cbar = colorbar(ax1.get_children()[0], cax=cax, orientation='vertical', format='%.0f%%')
#     cax.yaxis.set_ticks_position('left')
#     cbar.solids.set_edgecolor("face")
#     cbar.ax.tick_params(labelsize=11, length=4, pad=2)
#     ax1.set_title('% Attention', size=12)
#     ax1.set_xlabel('Head', size=11)
#     ax1.set_ylabel('Layer', size=11)
#     for _, spine in ax1.spines.items():
#         spine.set_visible(True)
#     ax2.set_title('Max', size=12)
#     bp = sns.barplot(x=layer_max * 100, ax=ax2, y=list(range(layer_max.shape[0])), color="#5a3dc4", orient="h",
#                      edgecolor="none")
#     formatter = FuncFormatter(lambda y, pos: '0' if (y == 0) else "%d%%" % (y))
#     ax2.xaxis.set_major_formatter(formatter)
#     plt.setp(bp.get_xticklabels(), fontsize=11)
#     bp.tick_params(axis='x', pad=1, length=3)
#     ax2.invert_yaxis()
#     ax2.set_yticklabels([])
#     ax2.spines['top'].set_visible(False)
#     ax2.spines['right'].set_visible(False)
#     ax2.spines['left'].set_visible(False)
#     ax2.xaxis.set_ticks_position('bottom')
#     ax2.axvline(0, linewidth=.85, color='black')

#     # plt.tight_layout()
#     plt.show()
#     fig_path = Path(dir_path) / f'{filename}.jpg'
#     fig.savefig(fig_path, bbox_inches='tight', pad_inches=0)
#     plt.close()


# def top_heads_attention_proportions(attentions_all_layers, proportion="attention", theta=0, min_total=0):
#     max_attentive_heads_pvalues = []
#     inv_class_labels_dict = {v:k for k, v in config_dict['CLASS_LABELS'].items()}
#     if theta == None:
#         raise ValueError("top_heads_attention_proportions: theta: got None, expected float")

#     top_heads_attention_pct = {}

#     for target_label in attentions_all_layers.keys():
#         attentions_sum = attentions_all_layers[target_label]
#         if inv_class_labels_dict[target_label] not in top_heads_attention_pct.keys():
#             top_heads_attention_pct[inv_class_labels_dict[target_label]] = {}
#         for domain in domain_coordinates_1based.keys():
#             domain_start_idx, domain_end_idx = convert_domain_coords_in_token_indices(domain)

#             # (1) the proportion of high-confidence attention arcs (αi,j > θ) for which f(i,j)=1 for top head
#             weight_domain_proportion_by_head, num_pos, num_total = calculate_weight_domain_proportion_by_head(attentions_sum, domain, proportion='high-attention-tokens-count', theta=theta)
#             exclude_mask = np.array(num_total) < min_total
#             masked_weight_domain_proportion_by_head = np.ma.masked_array(weight_domain_proportion_by_head, mask=exclude_mask)
#             top_head = np.unravel_index(np.argmax(masked_weight_domain_proportion_by_head, axis=None), masked_weight_domain_proportion_by_head.shape)
#             high_confidence_pct = masked_weight_domain_proportion_by_head[top_head[0],top_head[1]].item() * 100
#             num_pos = num_pos[top_head[0],top_head[1]].item()
#             num_total = num_total[top_head[0],top_head[1]].item()

#             # # (2) background: the proportion of all possible pairs i, j for which f(i,j)=1
#             if proportion == "attention" or proportion == "high-confidence-attention":
#                 weight_domain_count_by_head = attentions_sum[:, :, :, domain_start_idx:domain_end_idx+1].sum((2, 3))
#                 weight_total_count_by_head = attentions_sum.sum((2, 3))
#                 num_pos_background = weight_domain_count_by_head[top_head[0],top_head[1]].item()
#                 num_total_background = weight_total_count_by_head[top_head[0],top_head[1]].item()
#                 background_pct = num_pos_background / num_total_background * 100
#             elif proportion == "high-attention-tokens-count":
#                 mask_domain = torch.zeros(attentions_sum.shape, dtype=torch.bool)
#                 mask_domain[:, :, :, domain_start_idx:domain_end_idx+1] = 1
#                 weight_domain_count_by_head = mask_domain.long().sum((2, 3))
#                 mask_total = torch.ones(attentions_sum.shape, dtype=torch.bool)
#                 weight_total_count_by_head = mask_total.long().sum((2, 3))
#             else:
#                 raise ValueError("plt_attentions_domain: proportion must be one of %r." % ["attention", "high-attention-tokens"])

#             # p-value for statistical significance
#             stat, p_value = proportions_ztest(np.asarray([num_pos_background, num_pos]), np.asarray([num_total_background, num_total]))
#             m = masked_weight_domain_proportion_by_head.shape[0]*masked_weight_domain_proportion_by_head.shape[1]
#             alpha = 0.05 / m    # Bonferroni adjustment
#             statistical_significance = p_value < alpha
#             max_attentive_heads_pvalues.append([inv_class_labels_dict[target_label], domain, f'{top_head[0]+1}-{top_head[1]+1}', high_confidence_pct, background_pct, f'{p_value:.25f}', statistical_significance])
    
#             top_heads_attention_pct[inv_class_labels_dict[target_label]][domain] = high_confidence_pct

#     df_top_heads_attention_pct = pd.DataFrame(top_heads_attention_pct)
#     df_top_heads_attention_pct['MEDIAN'] = df_top_heads_attention_pct.median(axis=1)
#     df_max_attentive_heads_pvalues = pd.DataFrame(max_attentive_heads_pvalues, columns=['Class', 'Domain', 'Top head', 'Attn %', 'Background %', 'p-value', f'p<{alpha:.8f}'])

#     return df_top_heads_attention_pct.sort_values("MEDIAN", axis=0, ascending=False), df_max_attentive_heads_pvalues

# def show_attentions_for_each_domain(attentions_all_layers, dir_path, proportion="attention"):
#     inv_class_labels_dict = {v:k for k, v in config_dict['CLASS_LABELS'].items()}
#     if proportion == 'high-attention-tokens-count':
#         theta = 0.3
#     else:
#         theta = None

#     for target_label in attentions_all_layers.keys():
#         for domain in domain_coordinates_1based.keys():
#             plt_attentions_domain(attentions_all_layers[target_label], 
#                             domain, 
#                             dir_path, 
#                             filename=f"{domain}_{inv_class_labels_dict[int(target_label)]}", 
#                             proportion=proportion,
#                             theta=theta,
#                             title=f"Domain: {domain}, Class: {inv_class_labels_dict[int(target_label)]}")


def plt_attentions(mat, tick_labels, dir_path, filename="attention_matrix", theta=0, fig_size=(130,100), annot=False, cmap=sns.color_palette("viridis_r"), title=None):
    '''
    plot the NxN matrix passed as a heat map

    mat: square matrix to visualize
    tick_labels: labels for xticks and yticks (the tokens in our case)
    '''

    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"Directory '{dir_path}' created")

    #percentile_threshold = np.percentile(mat, percentile)
    # mask_att_below_thresh = mat < mat.max() * threshold_ratio
    # mask_att_above_thresh = mat >= mat.max() * threshold_ratio
    mask_att_below_thresh = mat < theta
    mask_att_above_thresh = mat >= theta
    
    xs, ys = np.where(mask_att_above_thresh == True) 
    high_attention_positions = []
    print(len(xs))
    print(len(ys))
    print(mat.shape)
    for x,y in zip(xs,ys):
        high_attention_positions.append([tick_labels[x],tick_labels[y],mat[x,y]])
    sorted_high_attention_positions = pd.DataFrame(high_attention_positions, columns=['Row','Col','Attn. Score']).sort_values(by=['Attn. Score'], ascending=False)
    with open(f"{Path(dir_path) / filename}.txt", 'a') as fp:
        fp.write(title+'\n')
        fp.write(sorted_high_attention_positions.to_string(index=False)) 
        fp.write('\n')

    fig, ax = plt.subplots(figsize=fig_size) 
    ax = sns.heatmap(mat, 
                     annot=annot,
                     yticklabels=tick_labels,
                     xticklabels=tick_labels, 
                     cmap=cmap,
                     mask=mask_att_below_thresh)   
    ax.xaxis.set_ticks_position('top')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=8)
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=8)
    xtick_above_threshold = [np.any(mask_att_above_thresh[:,i]) for i in range(0, len(mask_att_above_thresh))]
    ytick_above_threshold = [np.any(mask_att_above_thresh[i,:]) for i in range(0, len(mask_att_above_thresh))]
    for (is_above_threshold, ticklbl) in zip(xtick_above_threshold, ax.xaxis.get_ticklabels()):
        if is_above_threshold:
            # ticklbl.set_weight("bold")
            ticklbl.set(color = 'black', backgroundcolor='yellow', weight='bold', alpha=0.5)
        # ticklbl.set_color('blue' if is_above_threshold else 'black')
        # ticklbl.set_backgroundcolor('blue' if is_above_threshold else '0')
        # ticklbl.set(color = 'white' if is_above_threshold else 'black', backgroundcolor='blue' if is_above_threshold else 'white')
    for (is_above_threshold, ticklbl) in zip(ytick_above_threshold, ax.yaxis.get_ticklabels()):
        if is_above_threshold:
            ticklbl.set(color = 'black', backgroundcolor='yellow', weight='bold', alpha=0.5)
            # ticklbl.set_weight("bold")
    if title:
        ax.set_title(title, fontsize=80)
    else:
        ax.set_title('Attention matrix', fontsize=80)
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=80)

    # # padding_idx = n_tokens_per_seq + 2   # +2 to consider also 'CLS' and 'SEP' tokens
    # padding_idx = tick_labels.index(next(i for i in tick_labels if i.endswith('_[PAD]')))
    # ax.hlines([padding_idx], *ax.get_xlim(), linestyles='dashed')
    # ax.vlines([padding_idx], *ax.get_ylim(), linestyles='dashed')

    # plt.tight_layout()
    plt.show()

    # n_figures = len(os.listdir(dir_path))
    # if n_figures > 0:
    #     fig_path = Path(dir_path) / f'{filename}({n_figures}).jpg'
    # else:
    fig_path = Path(dir_path) / f'{filename}.jpg'
    fig.savefig(fig_path, bbox_inches='tight', pad_inches=0)


def plot_overlaid_attentions(dict_attentions, tick_labels, dir_path, threshold_ratio=0.5, fig_size=(80,80), annot=False, title=None, cmap=colormaps):
    '''
    Plot heatmap with all attention matrices of last layer of different classes
    superimposed with different colours 

    dict_attentions: square matrices to visualize
    path = path to save
    tick_labels: labels for xticks and yticks (the tokens in our case)
    '''
    fig, ax = plt.subplots(figsize=fig_size) 
    i = 0
    for label, mat in dict_attentions.items():
        #ax = sns.heatmap(mat, annot=annot, cmap=cmap[label])
        mask_att_below_thresh = mat < mat.max() * threshold_ratio
        ax = sns.heatmap(mat, 
                         annot=annot, 
                         mask=mask_att_below_thresh, 
                         cmap=cmap[int(label)], 
                         alpha=1-i/len(dict_attentions.keys()), #alpha=0.9, 
                         xticklabels=1, yticklabels=1,
                         square = True, 
                         cbar = False,
                        #  cbar_kws={"shrink":0.10, "label": f"{int(label)}"}
                         )
        i += 1

    ax.xaxis.set_ticks_position('top')
    ax.set_xticklabels(tick_labels, rotation=90)
    ax.set_yticklabels(tick_labels, rotation=0)
    # ax.xaxis.set_ticks_position('top')
    # ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    # mask_att_above_thresh = mat > mat.max() / 2

    # padding_idx = n_tokens_per_seq + 2   # +2 to consider also 'CLS' and 'SEP' tokens
    padding_idx = next(i for i in tick_labels if i.endswith('[PAD]'))
    ax.hlines([padding_idx], *ax.get_xlim(), linestyles='dashed')
    ax.vlines([padding_idx], *ax.get_ylim(), linestyles='dashed')

    if title:
        ax.set_title(title, fontsize=80)
    else:
        ax.set_title('Attention matrix', fontsize=80)

    # plt.tight_layout()
    plt.show()

    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"Directory '{dir_path}' created")

    fig_path = Path(dir_path) / f'{title}.jpg'
    fig.savefig(fig_path, bbox_inches='tight', pad_inches=0)


####Test functions

In [ ]:
def distance_cone(mat):
    # norm_per_row = np.linalg.norm(mat, axis=1)
    normalized_rows_mat =  mat/np.linalg.norm(mat, ord=2, axis=1, keepdims=True)
    # norm_per_row = np.linalg.norm(norm, ord=2, axis=1, keepdims=True) #math.sqrt(sum([i**2 for i in mat[0]]))
    # print(f'NORMA RIGhe = {norm_per_row}')
    # norm = [i/norm_per_row for i in mat[0]]
    # print(norm)

    # #print(root_sum_lambda_i)
    # norm_lambda = [abs(lambda_i)/root_sum_lambda_i for lambda_i in eigvals]
    # # print(norm_per_row)
    # normalized_rows_mat = []
    # for row in range(mat.shape[0]):
    #     normalized_rows_mat.append(mat[row, :] / norm_per_row[row])
    summed_rows_array = np.sum(normalized_rows_mat, axis=0)
    norm_summed_rows_mat = np.linalg.norm(summed_rows_array)
    return norm_summed_rows_mat

def singular_values(mat):
    return abs(np.linalg.svd(mat, compute_uv=False))

In [ ]:
def mat_save_csv(mat, name, dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
        print(f"Directory '{dirpath}' created")
    np.savetxt(Path(dirpath) / f'{name}.csv', mat, delimiter=",")

def chi2_test(multihead_output):
    pass
    #n_bins = 28
    # for row in range(multihead_output.shape[0]):
    #     # print(f'min={min(multihead_output[row,:])} | max={max(multihead_output[row,:])}')
    #     # print(multihead_output[row,:])
    #     bins = pd.cut(multihead_output[row,:], bins=n_bins)
    #     bin_counts = pd.value_counts(bins)
    #     print(bins.index.tolist())

    #     mean_val = []
    #     sum_bins = 0
    #     for i,bin_count in enumerate(bin_counts):
    #         bin_interval = bin_counts.index.tolist()[i]
    #         mv = (bin_interval.right-bin_interval.left)/2
    #         mean_val.append(mv)
    #         n = bin_count*mv
    #         sum_bins += n
    #     empirical_mean = sum_bins / 768
    #     mean_row = np.mean(multihead_output[row,:])
        
    #     sum_quadr_res = 0
    #     for i,bin_count in enumerate(bin_counts):
    #         mv = mean_val[i]
    #         sum_quadr_res += ((mv - empirical_mean)**2) * bin_count 
    #     empirical_stddev = math.sqrt(1/(768) * sum_quadr_res)
    #     stddev_row = np.std(multihead_output[row,:])
        
    #     exp_freq = []
    #     for i,bin_count in enumerate(bin_counts):
    #         exp_freq.append(1/(math.sqrt(2*math.pi) * empirical_stddev) * math.exp(-(mean_val[i]-empirical_mean)**2/(2*empirical_stddev**2)))
        
    #     chisq_test_stat, p_value = chisquare(bin_counts/768, exp_freq, n_bins-1)
    #     print(f'row {row} | chisq_test_stat ={chisq_test_stat} | p-value = {p_value}')

    #     # print(f'empirical_mean = {empirical_mean} | empirical_stddev = {empirical_stddev}')
    #     print(f'mean = {mean_row} | stddev = {stddev_row}')
    #     print(mean_val)
    #     plt.bar(mean_val, bin_counts/768, width=0.2)
    #     # plt.hist(multihead_output[row,:]/768, bins=n_bins)
    #     mu = mean_row
    #     variance = stddev_row**2
    #     sigma = stddev_row
    #     x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
    #     # plt.plot(x, stats.norm.pdf(x, mu, sigma))
    #     # mu = mean_row
    #     # variance = stddev_row**2
    #     # sigma = stddev_row
    #     # x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
    #     def f(y):
    #         return [1/(math.sqrt(2*math.pi) * stddev_row) * math.exp(-(yi-mean_row)**2/(2*stddev_row**2)) for yi in y]
    #     plt.plot(x, f(x), color='black')
    #     plt.show()

    #     break    

In [ ]:
def compute_Von_Neumann_entropy_eigvals(mat, head_lay):
    eigvals = np.linalg.eigvals(mat)

    # print(f'{head_lay} {len(eigvals) - np.count_nonzero(eigvals)}/{len(eigvals)}')
    # print(f'{head_lay} {eigvals}')

    # dir_path = math_interpret_dir
    # fig = plt.figure(figsize=(8,8))
    # plt.hist(eigvals)
    # plt.title(f"Histogram of eigenvalues of the attention matrix for {head_lay}")
    # plt.grid(linestyle = '--')
    # plt.yticks(list(plt.yticks()[0]) + [1])
    # plt.show()
    # dit_path_head = Path(dir_path) / 'hist_sing_val' / head_lay
    # if not os.path.exists(dit_path_head):
    #     os.makedirs(dit_path_head)
    # fig_path = Path(dit_path_head) / f'head_lay.jpg'
    # fig.savefig(fig_path, bbox_inches='tight')
    # plt.close()

    root_sum_lambda_i = math.sqrt(sum([abs(lambda_i)**2 for lambda_i in eigvals]))
    #print(root_sum_lambda_i)
    norm_lambda = [abs(lambda_i)/root_sum_lambda_i for lambda_i in eigvals]
    #print(norm_lambda)
    return -sum([abs(lambda_i_norm) * np.log(abs(lambda_i_norm)) for lambda_i_norm in norm_lambda if lambda_i_norm!=float(0)])

    # root_sum_lambda_i = math.sqrt(sum([abs(lambda_i)**2 for lambda_i in eigvals]))
    # #print(root_sum_lambda_i)
    # norm_lambda = [lambda_i/root_sum_lambda_i for lambda_i in eigvals]
    # #print(norm_lambda)
    # return -sum([lambda_i_norm * np.log(lambda_i_norm) for lambda_i_norm in norm_lambda if lambda_i_norm!=float(0)])

def compute_Shannon_entropy(mat, head_lay):
    
    bins = pd.cut(mat.ravel(), bins=20)
    shannon_entropy=0
    tot_count = len(mat.ravel())
    pjs = pd.value_counts(bins) / tot_count
    for pj in pjs:
        if pj != 0:
            shannon_entropy += pj*np.log(pj)

    return -shannon_entropy

def supervisedBertClassifierTest(input_embs, model, log_fp_test, theta=0, selected_layer_head_list=None, dir_path=None, title=""):
    gc.collect()
    torch.cuda.empty_cache()

    print('Calculating predictions...')
    log_fp_test.write("\nTest:\n")
    log_fp_test.write("=====\n")
    # Put model in evaluation mode
    model.eval()

    final_data_test = {'outputs' : [],
                  'logits' : [],
                  'output_embeddings' : [],
                  'input_embeddings' : [],
                  }
    attentions = {}
    attentions_last_layer = None # 1 heatmap per class, with attention scores of last layer of all samples 
    attentions_all_layers = None # 1 list of attention matrices per class, with attention scores of all layers of 1 sample
    attentions_all_layers_thresh = None
    repr_token_base_positions_axis = None

    distance_cones={}
    distance_cones_1_sample={}
    count_layer = {k:0 for k in range(-1, n_layers)}

    layer_query_weight_names = [f'bert.encoder.layer.{l}.attention.self.query.weight' for l in range(n_layers)]
    layer_key_weight_names = [f'bert.encoder.layer.{l}.attention.self.key.weight' for l in range(n_layers)]
    layer_value_weight_names = [f'bert.encoder.layer.{l}.attention.self.value.weight' for l in range(n_layers)]
    model_params = dict(model.named_parameters())

    # Predict 

    # Unpack the inputs from our dataloader
    input_embs = input_embs.to(device, dtype = torch.long)

    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        outputs = model(inputs_embeds=input_embs,
                    return_dict=True,
                    output_attentions=True,
                    output_hidden_states=True)        

    logits = outputs.logits

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    if config_dict['TASK_TYPE'] !='clustering':
        final_data_test['logits'].extend(outputs.logits.cpu().detach().numpy().tolist())
        # final_data_test['outputs'].extend(torch.softmax(outputs.logits).cpu().detach().numpy().tolist())
        if config_dict['N_CLASSES'] > 2:
            final_data_test['outputs'].extend(torch.softmax(outputs.logits, dim=1).cpu().detach().numpy().tolist())
        else:
            final_data_test['outputs'].extend(torch.sigmoid(outputs.logits).cpu().detach().numpy().tolist())

    #############################################   
    sample_idx = 0

    if config_dict['TASK_TYPE']=='print_layer_output':
        input_embs = outputs.hidden_states[0][sample_idx].cpu().detach().numpy() # X = imput to current layer (output of previous layer)
                                                                                                # NB: hidden_states of attn layers shifted by +1 because hidden_states[0] is embedding layer
        mat_save_csv(input_embs, f'input_embeddings', math_interpret_dir)
        return final_data_test, attentions      

    # get attention matrices
    # NB:   outputs.attentions is a tuple containing 12 elements, i.e. attentions of the 12 layers of Bert 
    #       eg. if we consider first layer from the bottom (closest to the input):
    #           print(outputs.attentions[0].shape) 
    #               -> torch.Size([4, 12, 512, 512]) -> [batch_size , num_heads, seq_len, seq_len]
    elif config_dict['TASK_TYPE']=='attention_analysis' or config_dict['TASK_TYPE']=='attention_flow':

        # get attention scores of sample from all heads in all layers
        sample_attentions = []
        sample_attentions_thresh = []
        for layer in range(n_layers):
            layer_attentions = []
            masked_layer_attentions = []
            for head in range(n_heads):
                # Source: "Bertology meets Biology...": We exclude attention to the [SEP] delimiter token, as it has been shown to 
                # be a “no-op” attention token (Clark et al., 2019), as well as attention to 
                # the [CLS] token, which is not explicitly used in language modeling.
                # Additionally, exclude attention on [PAD] tokens.
                head_attentions = get_attentions(outputs.attentions, sample_idx_in_batch=sample_idx, layer=layer, attention_head=head, sum=False)
                layer_attentions.append(head_attentions)
                mask_att_below_thresh = head_attentions < theta
                masked_head_attentions = np.ma.masked_array(head_attentions, mask=mask_att_below_thresh, fill_value=0)
                masked_layer_attentions.append(masked_head_attentions.filled())
            sample_attentions.append(layer_attentions)
            sample_attentions_thresh.append(masked_layer_attentions)
        
        # update sum of class attention scores of all heads of all layers
        attentions_all_layers = torch.zeros(np.asarray(sample_attentions).shape, dtype=torch.double)
        repr_token_base_positions_axis = []
        for n in range(config_dict['MAX_LENGTH']):
            if n==0:
                repr_token_base_positions_axis.append('[CLS]')
            else:
                repr_token_base_positions_axis.append(f"{n+1}") 
        # repr_token_base_positions_axis [target_label] = [f"{n*config_dict['STRIDE']-config_dict['STRIDE']}_{tokenizer.convert_ids_to_tokens(id)}" for n,id in enumerate(ids[sample_idx].cpu().detach().numpy().tolist())]
        
        attentions_all_layers_thresh = torch.zeros(np.asarray(sample_attentions_thresh).shape, dtype=torch.double)
        
        for layer in range(n_layers):
            for head in range(n_heads):
                attentions_all_layers[layer][head] = attentions_all_layers[layer][head] + np.asarray(sample_attentions[layer][head])
                attentions_all_layers_thresh[layer][head] = attentions_all_layers_thresh[layer][head] + np.asarray(sample_attentions_thresh[layer][head])

    # elif config_dict['TASK_TYPE']=='check_normality':
    #     for sample_idx in range(0,len(seq_ids)):
    #         target_label = label_ids[sample_idx]
    #         name_seq=seq_ids[sample_idx]
    #         if target_label == np.argmax(logits[sample_idx]): # correct prediction
    #             X = outputs.hidden_states[0][sample_idx].cpu().detach().numpy()  # X = imput to current layer (output of previous layer)
    #                                                                                     # NB: hidden_states of attn layers shifted by +1 because hidden_states[0] is embedding layer
    #                                                                                     # token embeddings shape=(n_tokens_in_seq, emb_dim)=(512, 768)

    #             for i, row in enumerate(X):
    #                 print(f'Mean row {i}: {np.mean(row)}')  
    #         break
                
    
    elif config_dict['TASK_TYPE']=='distance_cones_analysis':
        for layer in set(np.asarray(selected_layer_head_list)[:,0]):
            X = outputs.hidden_states[layer][sample_idx].cpu().detach().numpy()  # X = imput to current layer (output of previous layer)
                                                                                # NB: hidden_states of attn layers shifted by +1 because hidden_states[0] is embedding layer
                                                                                # token embeddings shape=(n_tokens_in_seq, emb_dim)=(512, 768)
            # print(f'-------NORMA X[0]: {np.linalg.norm(X[0])}')
            rows_sum_X = np.sum(X, axis=0)
            # norm = np.linalg.norm(rows_sum_X, ord=2)
            # print(f'-------NORMA X: {norm}')
            # model_params = dict(model.named_parameters())
            # print(model_params["bert.encoder.layer.0.attention.self.value.weight"].data)
            W_v = model_params[f"bert.encoder.layer.{layer}.attention.self.value.weight"].data.cpu().detach().numpy()
            W_q = model_params[f"bert.encoder.layer.{layer}.attention.self.query.weight"].data.cpu().detach().numpy()
            W_k = model_params[f"bert.encoder.layer.{layer}.attention.self.key.weight"].data.cpu().detach().numpy()
            emb_dim = X.shape[1]
            d_k = int(emb_dim / n_heads)
            multihead_output = None

            for head in np.sort([h for l, h in selected_layer_head_list if l==layer]):
                W_v_i = W_v[:, head*d_k : head*d_k + d_k]
                W_q_i = W_q[:, head*d_k : head*d_k + d_k]
                W_k_i = W_k[:, head*d_k : head*d_k + d_k]
                V_i = np.matmul(X, W_v_i)
                Q_i = np.matmul(X, W_q_i)
                K_i = np.matmul(X, W_k_i)

                softmax_i = torch.softmax(torch.tensor(np.matmul(Q_i, K_i.T) / math.sqrt(d_k)), dim=1).cpu().detach().numpy()
                output_head_i = np.matmul(softmax_i, V_i)

                if head == 5-1:
                    output_head_5 = np.copy(output_head_i)

                if multihead_output is None:
                    multihead_output = np.copy(output_head_i)
                else:
                    multihead_output = np.hstack((multihead_output, output_head_i))

            layerNorm = nn.LayerNorm(model.config.hidden_size, eps=model.config.layer_norm_eps)
            multihead_output = layerNorm(torch.from_numpy(X + multihead_output)).cpu().detach().numpy()

            layer_output = outputs.hidden_states[layer+1][sample_idx].cpu().detach().numpy() # output of current layer

        #     print(f'Saving mat L{layer+1}')
        #     dirpath = Path(math_interpret_dir) / 'Y_outputs'
        #     mat_save_csv(multihead_output, f'Y_output_L{layer+1}', dirpath)

        # return final_data_test, test_accuracies, attentions, timings
            
            if layer not in distance_cones:
                distance_cones[layer] = {}

            dc_X = distance_cone(X)
            # print(f'-------({name_seq}) DISTANCE CONE X: {dc_X}')
            if 'Layer input' not in distance_cones[layer]:
                distance_cones[layer]['Layer input'] = []
            distance_cones[layer]['Layer input'].append(dc_X)

            dc_output_head_5 = distance_cone(output_head_5)
            # print(f'-------({name_seq}) DISTANCE CONE output_head_5: {dc_output_head_5}')
            if 'Head output' not in distance_cones[layer]:
                distance_cones[layer]['Head output'] = []
            distance_cones[layer]['Head output'].append(dc_output_head_5)

            dc_multihead_output = distance_cone(multihead_output)
            # print(f'-------({name_seq}) DISTANCE CONE multihead_i: {dc_multihead_output}')
            if 'Multihead output' not in distance_cones[layer]:
                distance_cones[layer]['Multihead output'] = []
            distance_cones[layer]['Multihead output'].append(dc_multihead_output)

            dc_layer_output = distance_cone(layer_output)
            # print(f'-------({name_seq}) DISTANCE CONE layer_output: {layer_output}')
            if 'Layer output' not in distance_cones[layer]:
                distance_cones[layer]['Layer output'] = []
            distance_cones[layer]['Layer output'].append(dc_layer_output)

            if count_layer[layer] == 0:
                if layer == 0:
                    distance_cones_1_sample[-1] = dc_X
                distance_cones_1_sample[layer] = dc_layer_output
           
                            

    # elif config_dict['TASK_TYPE']=='layer_output_analysis':     
    #     output_layers_dir = Path(math_interpret_dir) / "output_layer_histograms"
    #     if not os.path.exists(output_layers_dir):
    #         os.makedirs(output_layers_dir)
    #         print(f"Directory '{output_layers_dir}' created")

    #     for sample_idx in range(0,len(seq_ids)):
    #         target_label = label_ids[sample_idx]
    #         name_seq=seq_ids[sample_idx]
            
    #         if target_label == config_dict['CLASS_LABELS']['omicron'] and target_label == np.argmax(logits[sample_idx]): # correct prediction

    #             # output layer histogram:
    #             for layer in set(np.asarray(selected_layer_head_list)[:,0]):
    #                 layer_output = outputs.hidden_states[layer+1][sample_idx].cpu().detach().numpy()
    #                 layer_ouput_sum_rows = np.sum(layer_output, axis=0)
    #                 fig = plt.figure(figsize=(8,8))
    #                 plt.hist(layer_ouput_sum_rows, bins=27)
    #                 plt.title(f"L{layer+1} output")
    #                 plt.grid(linestyle = '--')
    #                 plt.yticks(list(plt.yticks()[0]) + [1])
    #                 plt.show()
    #                 fig_path = Path(output_layers_dir) / f'output_layer_{layer+1}.jpg'
    #                 fig.savefig(fig_path)
    #         return final_data_test, test_accuracies, attentions, timings


    elif config_dict['TASK_TYPE']=='singularvalues_ratio_analysis':
        mat_ratio_max_min_sv_softmax_i = np.zeros((n_layers,n_heads))
        mat_ratio_2ndmin_min_sv_softmax_i = np.zeros((n_layers,n_heads))
        mat_ratio_max_min_sv_W_v_i = np.zeros((n_layers,n_heads))
        mat_ratio_2ndmin_min_sv_W_v_i = np.zeros((n_layers,n_heads))
        mat_count_zero_sv_softmax_i = np.zeros((n_layers,n_heads))
        mat_count_zero_sv_W_v_i = np.zeros((n_layers,n_heads))
        histograms = {}

        dir_path_sv = Path(math_interpret_dir) / f'sing_vals_{title}'
        if not os.path.exists(dir_path_sv):
            os.makedirs(dir_path_sv)
        singvals_ratio_file = Path(dir_path_sv) / "singvals_ratio.txt"

        for layer, head in selected_layer_head_list:

            X = outputs.hidden_states[layer+1][sample_idx].cpu().detach().numpy() # token embeddings shape=(n_tokens_in_seq, emb_dim)=(512, 768)
            emb_dim = X.shape[1]
            d_k = int(emb_dim / n_heads)
            W_q = model_params[layer_query_weight_names[layer]].data.cpu().detach().numpy()
            W_k = model_params[layer_key_weight_names[layer]].data.cpu().detach().numpy()
            W_v = model_params[layer_value_weight_names[layer]].data.cpu().detach().numpy()
            W_q_i = W_q[:, head*d_k : head*d_k + d_k]
            W_k_i = W_k[:, head*d_k : head*d_k + d_k]
            W_v_i = W_v[:, head*d_k : head*d_k + d_k]

            Q_i = np.matmul(X, W_q_i)
            K_i = np.matmul(X, W_k_i)
            V_i = np.matmul(X, W_v_i)

            # Qi_KiT = np.matmul(Q_i, K_i.T)
            softmax_i = torch.softmax(torch.tensor(np.matmul(Q_i, K_i.T) / math.sqrt(d_k)), dim=1).cpu().detach().numpy()
            singular_values_softmax_i = singular_values(softmax_i)
            mat_ratio_max_min_sv_softmax_i[layer][head] = max(singular_values_softmax_i) / min(singular_values_softmax_i)
            secondmin_sv_softmax_i = min(np.delete(singular_values_softmax_i, singular_values_softmax_i.argmin()))
            mat_ratio_2ndmin_min_sv_softmax_i[layer][head] = secondmin_sv_softmax_i / min(singular_values_softmax_i)

            singular_values_W_v_i = singular_values(W_v_i)
            mat_ratio_max_min_sv_W_v_i[layer][head] = max(singular_values_W_v_i) / min(singular_values_W_v_i)
            secondmin_sv_W_v_i = min(np.delete(singular_values_W_v_i, singular_values_W_v_i.argmin()))
            mat_ratio_2ndmin_min_sv_W_v_i[layer][head] = secondmin_sv_W_v_i / min(singular_values_W_v_i)

            mat_count_zero_sv_softmax_i[layer][head] = (singular_values_softmax_i < 0.01).sum()/len(singular_values_softmax_i)
            mat_count_zero_sv_W_v_i[layer][head] = (singular_values_W_v_i < 0.5).sum()/len(singular_values_W_v_i)

            def generate_hist_sv(sv, h, l, title, histograms):
                counts, bins = np.histogram(sv, 10)
                if title not in histograms.keys():
                    histograms[title] = {}
                if h not in histograms[title].keys():
                    histograms[title][h] = []
                histograms[title][h].append({
                    "label": f'Layer {l+1}\n(max SV: {max(sv)})',
                    "counts": counts,
                    "bins": bins,
                })
                return histograms


            if head in [0,4,8,9]:
                histograms = generate_hist_sv(singular_values_softmax_i, head, layer, "softmax", histograms)
                histograms = generate_hist_sv(singular_values_W_v_i, head, layer, "W_v", histograms)

        def save_to_file(mat, fp, title, short_title, fmt, path_fig):
            df_cols = [f"H{x+1}" for x in range(n_heads)]
            df_idx = [f"L{x+1}" for x in range(n_layers)]
            df = pd.DataFrame(mat, columns=df_cols)
            df.insert(0, "Layer\Head", df_idx)
            df_tabulate = tabulate(df, headers=df.columns, showindex=False, stralign='right', floatfmt=fmt)
            print(title)
            print("=================================================================")
            print(df_tabulate)
            print()
            fp.write(f'{title}\n=================================================================\n')
            fp.write(df_tabulate)
            fp.write("\n\n")
            df.set_index("Layer\Head")
            fig = plt.figure(figsize=(15,15))
            plt.title(title)
            sns.heatmap(mat, annot=True, fmt=fmt, yticklabels=df_idx, xticklabels=df_cols)
            plt.show()
            fig.savefig(Path(path_fig) / f'heatmap_{short_title}.jpg', bbox_inches='tight')
            plt.close()


        with open(singvals_ratio_file, 'w') as singvals_ratio_fp:
            save_to_file(mat_ratio_max_min_sv_softmax_i, singvals_ratio_fp, "Ratio between the highest and the lowest singular value of matrix softmax((QK^T)/d_k)", "ratio_max_min_sv_softmax", '.2e', dir_path_sv)
            save_to_file(mat_ratio_2ndmin_min_sv_softmax_i, singvals_ratio_fp, "Ratio between the second lowest and the lowest singular value of matrix softmax((QK^T)/d_k)", "ratio_2ndmin_min_sv_softmax", '.2f', dir_path_sv)
            save_to_file(mat_ratio_max_min_sv_W_v_i, singvals_ratio_fp, "Ratio between the highest and the lowest singular value of matrix W_v", "ratio_max_min_sv_Wv", '.2f', dir_path_sv)
            save_to_file(mat_ratio_2ndmin_min_sv_W_v_i, singvals_ratio_fp, "Ratio between the second lowest and the lowest singular value of matrix W_v", "ratio_2ndmin_min_sv_Wv", '.2f', dir_path_sv)
            save_to_file(mat_count_zero_sv_softmax_i, singvals_ratio_fp, "Percentage of singular values < 0.01 of matrix softmax((QK^T)/d_k)", "count_zero_sv_softmax", ".2%", dir_path_sv)
            save_to_file(mat_count_zero_sv_W_v_i, singvals_ratio_fp, "Percentage of singular values < 0.5 of matrix W_v", "count_zero_sv_Wv", ".2%", dir_path_sv)

        def plot_hist_sv(hist_dict, title, path):
            for h, hist_list_h in hist_dict.items():
                fig,ax = plt.subplots(figsize=(8,8))
                plt.title(f"Histogram of Singular Values of {title} for head {h+1}")
                ax.set_xlabel("Singular Value")
                ax.set_ylabel("Frequency")
                plt.grid()
                colors = [cm.get_cmap("nipy_spectral")(i) for i in np.linspace(0, 1, n_layers)]
                ax.set_prop_cycle(cycler('color', colors))
                for hist in hist_list_h:
                    ax.step(hist['bins'][:-1], hist['counts'], label=hist['label'])
                plt.legend(bbox_to_anchor=(1.04, 1),loc='upper left')
                plt.show()
                fig_path = Path(path) / f'sing_val_{title}_{h+1}.jpg'
                fig.savefig(fig_path, bbox_inches='tight')
                plt.close()
        
        plot_hist_sv(histograms["softmax"], "softmax", dir_path_sv)
        plot_hist_sv(histograms["W_v"], "W_v", dir_path_sv)
                
        return final_data_test, attentions

    # elif config_dict['TASK_TYPE']=='eigenvalues_analysis':
        
    #     for sample_idx in range(0,len(seq_ids)):
    #         target_label = label_ids[sample_idx]

    #         if target_label == config_dict['CLASS_LABELS'][selected_class] and target_label == np.argmax(logits[sample_idx]): # correct prediction
    #             Qi_KiT_eigvals_file = Path(math_interpret_dir) / "von_neumann_entropy_Qi_KiT.txt"
    #             symm_comp_eigvals_file = Path(math_interpret_dir) / "von_neumann_entropy_symm_comp.txt"

    #             with open(Qi_KiT_eigvals_file, 'w') as Qi_KiT_eigvals_fp, open(symm_comp_eigvals_file, 'w') as symm_comp_eigvals_fp:
    #                 # get input embedding of sample for selected heads (shape=(n_layers, batch_dim, n_tokens_in_seq, emb_dim))
    #                 VN_entropy_Qi_KiT_layer_sum = np.zeros(n_layers)
    #                 VN_entropy_symm_comp_layer_sum = np.zeros(n_layers)
    #                 Sh_entropy_Qi_KiT_layer_sum = np.zeros(n_layers)
    #                 Sh_entropy_symm_comp_layer_sum = np.zeros(n_layers)
    #                 count_heads_layer = np.zeros(n_layers)

    #                 for layer, head in selected_layer_head_list:

    #                     X = outputs.hidden_states[layer+1][sample_idx].cpu().detach().numpy() # token embeddings shape=(n_tokens_in_seq, emb_dim)=(512, 768)
    #                     emb_dim = X.shape[1]
    #                     d_k = int(emb_dim / n_heads)
    #                     W_q = model_params[layer_query_weight_names[layer]].data.cpu().detach().numpy()
    #                     W_k = model_params[layer_key_weight_names[layer]].data.cpu().detach().numpy()
    #                     W_v = model_params[layer_value_weight_names[layer]].data.cpu().detach().numpy()
    #                     W_q_i = W_q[:, head*d_k : head*d_k + d_k]
    #                     W_k_i = W_k[:, head*d_k : head*d_k + d_k]
    #                     W_v_i = W_v[:, head*d_k : head*d_k + d_k]

    #                     Q_i = np.matmul(X, W_q_i)
    #                     K_i = np.matmul(X, W_k_i)
    #                     V_i = np.matmul(X, W_v_i)
                        
    #                     norm_Q_i = np.linalg.norm(Q_i, axis=1)
    #                     norm_K_i = np.linalg.norm(K_i, axis=1)
    #                     norm_V_i = np.linalg.norm(V_i, axis=1)

    #                     singular_values_W_q_i = abs(np.linalg.svd(W_q_i, compute_uv=False))
    #                     singular_values_W_k_i = abs(np.linalg.svd(W_k_i, compute_uv=False))
    #                     singular_values_W_v_i = abs(np.linalg.svd(W_v_i, compute_uv=False))
    #                     max_sv_W_q_i = max(singular_values_W_q_i)
    #                     max_sv_W_k_i = max(singular_values_W_k_i)
    #                     max_sv_W_v_i = max(singular_values_W_v_i)


    #                     softmax_i = torch.softmax(torch.tensor(np.matmul(Q_i, K_i.T) / math.sqrt(d_k)), dim=1).cpu().detach().numpy()
    #                     output_head_i = np.matmul(softmax_i, V_i)
    #                     norm_output_head_i = np.linalg.norm(output_head_i, axis=1)
    #                     singular_values_output_head_i = abs(np.linalg.svd(output_head_i, compute_uv=False))
    #                     max_sv_output_head_i = max(singular_values_output_head_i)

    #                     dit_path_head = Path(dir_path) / 'norm_Qi_Ki_Vi' / f'{layer+1}_{head+1}'
    #                     if not os.path.exists(dit_path_head):
    #                         os.makedirs(dit_path_head)

    #                     # # histograms of singular values
    #                     # fig = plt.figure(figsize=(8,8))
    #                     # plt.hist(singular_values_W_k_i)
    #                     # plt.title(f"Histogram of singular values of W_k_i for head {head+1} in layer {layer+1}")
    #                     # plt.grid(linestyle = '--')
    #                     # plt.yticks(list(plt.yticks()[0]) + [1])
    #                     # plt.show()
    #                     # dit_path_head = Path(dir_path) / 'hist_sing_val' / f'{layer+1}_{head+1}'
    #                     # if not os.path.exists(dit_path_head):
    #                     #     os.makedirs(dit_path_head)
    #                     # fig_path = Path(dit_path_head) / f'{layer+1}_{head+1}.jpg'
    #                     # fig.savefig(fig_path, bbox_inches='tight')
    #                     # plt.close()

    #                     # # histograms of norms of Q_i
    #                     # fig,ax = plt.subplots(figsize=(8,8))
    #                     # plt.hist(norm_Q_i)
    #                     # plt.title(f"Histogram of the norms of Q_i for head {head+1} in layer {layer+1}\n(max SV of W_q_i: {max_sv_W_q_i})")
    #                     # plt.grid(linestyle = '--')
    #                     # ax.set_xlabel("Norm of Q_i")
    #                     # ax.set_ylabel("Frequency")
    #                     # plt.show()
    #                     # fig_path = Path(dit_path_head) / f'norm_Qi_{layer+1}_{head+1}.jpg'
    #                     # fig.savefig(fig_path, bbox_inches='tight')
    #                     # plt.close()

    #                     # # histograms of norms of K_i
    #                     # fig,ax = plt.subplots(figsize=(8,8))
    #                     # plt.hist(norm_K_i)
    #                     # plt.title(f"Histogram of the norms of K_i for head {head+1} in layer {layer+1}\n(max SV of W_k_i: {max_sv_W_k_i})")
    #                     # plt.grid(linestyle = '--')
    #                     # ax.set_xlabel("Norm of K_i")
    #                     # ax.set_ylabel("Frequency")
    #                     # plt.show()
    #                     # fig_path = Path(dit_path_head) / f'norm_Ki_{layer+1}_{head+1}.jpg'
    #                     # fig.savefig(fig_path, bbox_inches='tight')
    #                     # plt.close()

    #                     # # histograms of norms of V_i
    #                     # fig,ax = plt.subplots(figsize=(8,8))
    #                     # plt.hist(norm_V_i)
    #                     # plt.title(f"Histogram of the norms of V_i for head {head+1} in layer {layer+1}\n(max SV of W_v_i: {max_sv_W_v_i})")
    #                     # plt.grid(linestyle = '--')
    #                     # ax.set_xlabel("Norm of V_i")
    #                     # ax.set_ylabel("Frequency")
    #                     # plt.show()
    #                     # fig_path = Path(dit_path_head) / f'norm_Vi_{layer+1}_{head+1}.jpg'
    #                     # fig.savefig(fig_path, bbox_inches='tight')
    #                     # plt.close()


    #                     # # histograms of norms of output_head_i
    #                     # fig,ax = plt.subplots(figsize=(8,8))
    #                     # plt.hist(norm_output_head_i)
    #                     # plt.title(f"Histogram of the norms of Y_i output for head {head+1} in layer {layer+1}\n(max SV of Y_i: {max_sv_output_head_i})")
    #                     # plt.grid(linestyle = '--')
    #                     # ax.set_xlabel("Norm of Y_i")
    #                     # ax.set_ylabel("Frequency")
    #                     # plt.show()
    #                     # fig_path = Path(dit_path_head) / f'norm_Yi_{layer+1}_{head+1}.jpg'
    #                     # fig.savefig(fig_path, bbox_inches='tight')
    #                     # plt.close()


    #                     # histogram of singular values of output_head_i
    #                     fig,ax = plt.subplots(figsize=(8,8))
    #                     plt.hist(singular_values_output_head_i)
    #                     plt.title(f"Histogram of Singular Values of Y_i output for head {head+1} in layer {layer+1}\n(max SV of Y_i: {max_sv_output_head_i})")
    #                     plt.grid(linestyle = '--')
    #                     ax.set_xlabel("Norm of Y_i")
    #                     ax.set_ylabel("Frequency")
    #                     plt.show()
    #                     fig_path = Path(dit_path_head) / f'sing_val_Yi_{layer+1}_{head+1}.jpg'
    #                     fig.savefig(fig_path, bbox_inches='tight')
    #                     plt.close()


    #                 #     # Von Neumann norms
    #                 #     count_heads_layer[layer] += 1
    #                 #     # Eigenvalues of QK^T
    #                 #     Qi_KiT = np.matmul(Q_i, K_i.T)
    #                 #     eigvals_Qi_KiT = np.linalg.eigvals(Qi_KiT)
    #                 #     n_eigvals_zero_Qi_KiT = f"{len(eigvals_Qi_KiT) - np.count_nonzero(eigvals_Qi_KiT)}/{len(eigvals_Qi_KiT)}"
    #                 #     # von neumann entropy and shannon entropy
    #                 #     VN_entropy_Qi_KiT = compute_Von_Neumann_entropy_eigvals(Qi_KiT, f"head{head+1}_layer_{layer+1}")
    #                 #     Sh_entropy_Qi_KiT = compute_Shannon_entropy(Qi_KiT, f"head{head+1}_layer_{layer+1}")
    #                 #     Qi_KiT_eigvals_fp.write(f"\thead_{head+1}_layer_{layer+1} | n_eigv_zero={n_eigvals_zero_Qi_KiT} | VN_entropy_={VN_entropy_Qi_KiT}\n")
    #                 #     VN_entropy_Qi_KiT_layer_sum[layer] += VN_entropy_Qi_KiT
    #                 #     Sh_entropy_Qi_KiT_layer_sum[layer] += Sh_entropy_Qi_KiT

    #                 #     # Eigenvalues of symmetric component
    #                 #     symm_comp_mat = (Qi_KiT + Qi_KiT.T) / 2
    #                 #     eigvals_symm_comp_mat = np.linalg.eigvals(symm_comp_mat)
    #                 #     n_eigvals_zero_symm_comp_mat = f"{len(eigvals_symm_comp_mat) - np.count_nonzero(eigvals_symm_comp_mat)}/{len(eigvals_symm_comp_mat)}"
    #                 #     # von neumann entropy
    #                 #     VN_entropy_symm_comp_mat = compute_Von_Neumann_entropy_eigvals(symm_comp_mat, f"head{head+1}_layer_{layer+1}")
    #                 #     Sh_entropy_symm_comp_mat = compute_Shannon_entropy(symm_comp_mat, f"head{head+1}_layer_{layer+1}")
    #                 #     symm_comp_eigvals_fp.write(f"\thead_{head+1}_layer_{layer+1} | n_eigv_zero={n_eigvals_zero_symm_comp_mat} | VN_entropy_head_{head+1}layer_{layer+1} = {VN_entropy_symm_comp_mat}\n")
    #                 #     VN_entropy_symm_comp_layer_sum[layer] += VN_entropy_symm_comp_mat
    #                 #     Sh_entropy_symm_comp_layer_sum[layer] += Sh_entropy_symm_comp_mat

    #                 # VN_entropy_Qi_KiT_layer_mean = VN_entropy_Qi_KiT_layer_sum / count_heads_layer
    #                 # VN_entropy_symm_comp_layer_mean = VN_entropy_symm_comp_layer_sum / count_heads_layer
    #                 # Sh_entropy_Qi_KiT_layer_mean = Sh_entropy_Qi_KiT_layer_sum / count_heads_layer
    #                 # Sh_entropy_symm_comp_layer_mean = Sh_entropy_symm_comp_layer_sum / count_heads_layer
    #                 # fig = plt.figure(figsize=(10,10))
    #                 # x = [i+1 for i in range(n_layers)]
    #                 # fig, ax = plt.subplots()
    #                 # ax.plot(x, VN_entropy_Qi_KiT_layer_mean, 'b', label='VN entropy of QiKi^T')
    #                 # ax.plot(x, VN_entropy_symm_comp_layer_mean, 'r', label='VN entropy of QiKi^T symm. comp.')
    #                 # ax.axis('equal')
    #                 # leg = ax.legend()
    #                 # plt.title(f"Average Von Neumann entropy for each layer")
    #                 # plt.grid(linestyle = '--')
    #                 # ax.set_xticks(x)
    #                 # #fig.yticks(list(plt.yticks()[0]) + [1])
    #                 # ax.set_xlabel("Layer")
    #                 # ax.set_ylabel("VN entropy")
    #                 # plt.show()
    #                 # fig_path = Path(dir_path) / f'VN_entropy_plot.jpg'
    #                 # fig.savefig(fig_path, bbox_inches='tight')
    #                 # plt.close()

    #                 # fig = plt.figure(figsize=(10,10))
    #                 # x = [i+1 for i in range(n_layers)]
    #                 # fig, ax = plt.subplots()
    #                 # ax.plot(x, Sh_entropy_Qi_KiT_layer_mean, 'b', label='Shannon entropy of QiKi^T')
    #                 # ax.plot(x, Sh_entropy_symm_comp_layer_mean, 'r', label='Shannon entropy of QiKi^T symm. comp.')
    #                 # ax.axis('equal')
    #                 # leg = ax.legend()
    #                 # plt.title(f"Average Shannon entropy for each layer")
    #                 # plt.grid(linestyle = '--')
    #                 # ax.set_xticks(x)
    #                 # #fig.yticks(list(plt.yticks()[0]) + [1])
    #                 # ax.set_xlabel("Layer")
    #                 # ax.set_ylabel("Shannon entropy")
    #                 # plt.show()
    #                 # fig_path = Path(dir_path) / f'Shannon_entropy_plot.jpg'
    #                 # fig.savefig(fig_path, bbox_inches='tight')
    #                 # plt.close()

    #             return final_data_test, test_accuracies, attentions, timings

    # if config_dict['TASK_TYPE']=='attention_analysis' or config_dict['TASK_TYPE']=='attention_flow':
    #     # calculate mean of attention matrices:
    #     for target_label in config_dict['CLASS_LABELS'].values():
    #         for layer in range(len(attentions_all_layers[target_label])):
    #             for head in range(len(attentions_all_layers[target_label][layer])):
    #                 attentions_all_layers[target_label][layer][head] = attentions_all_layers[target_label][layer][head] / count_class_samples[target_label]


    if config_dict['TASK_TYPE']=='distance_cones_analysis':
        dist_cones_path = Path(math_interpret_dir) / 'direction_cones'
        if not os.path.exists(dist_cones_path):
            os.makedirs(dist_cones_path)
        
        fig, ax = plt.subplots(figsize=((7,4)))
        ax.plot(list(distance_cones_1_sample.keys()), list(distance_cones_1_sample.values()), '-o')
        ax.set_title('Cone index')
        ax.set_xlabel("Layer")
        ax.set_ylabel("Index")
        ax.set_xticks(list(distance_cones_1_sample.keys()))
        ax.set_xticklabels(np.asarray(list(distance_cones_1_sample.keys()))+1)
        ax.grid()
        plt.show()
        fig_path = Path(dist_cones_path) / f'cone_index_{title}.jpg'
        fig.savefig(fig_path)
        fig.clear()

        # for layer, layer_distance_cones in distance_cones.items():
        #     n_bins=20
        #     distance_cones_1 = {k:v for k,v in layer_distance_cones.items() if k in ['Layer input', 'Multihead output', 'Layer output']}
        #     fig, ax = plt.subplots(figsize=((7,4)))
        #     ax.hist(distance_cones_1.values(), n_bins, histtype='step', stacked=True, fill=False, label=layer_distance_cones.keys())
        #     ax.set_title(f'L{layer} cone index')
        #     ax.legend(prop={'size': 10})
        #     ax.set_xlabel("Cone index")
        #     ax.set_ylabel("Frequency")
        #     box = ax.get_position()
        #     ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        #     ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
        #     plt.show()
        #     fig_path = Path(dist_cones_path) / f'direction_cones_L{layer}_1.jpg'
        #     fig.savefig(fig_path)
        #     fig.clear()

        #     distance_cones_2 = {k:v for k,v in layer_distance_cones.items() if k in ['Layer input', 'Head output']}
        #     fig, ax = plt.subplots(figsize=((7,4)))
        #     ax.hist(distance_cones_2.values(), n_bins, histtype='step', stacked=True, fill=False, label=layer_distance_cones.keys())
        #     ax.set_title(f'L{layer} cone index')
        #     ax.legend(prop={'size': 10})
        #     ax.set_xlabel("Cone index")
        #     ax.set_ylabel("Frequency")
        #     box = ax.get_position()
        #     ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        #     ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
        #     plt.show()
        #     fig_path = Path(dist_cones_path) / f'direction_cones_L{layer}_2.jpg'
        #     fig.savefig(fig_path)
        #     fig.clear()


    print('DONE.')

    # token_base_positions_axis = [f"{i*config_dict['STRIDE']-config_dict['STRIDE']+spike_gene_start}" for i in range(0,config_dict['MAX_LENGTH'])]

    attentions = {'attentions_all_layers' : attentions_all_layers,
                  'attentions_all_layers_thresh' : attentions_all_layers_thresh,
                  'repr_token_base_positions_axis' : repr_token_base_positions_axis
                  }

    return final_data_test, attentions

def show_test_plots(accuracies):
    # Create a barplot showing the accuracy score for each batch of test samples.
    fig = plt.figure(figsize=(12,6))
    ax = sns.lineplot(x=list(range(len(accuracies))), y=accuracies, ci=None)

    plt.title('Accuracy per Batch')
    plt.ylabel('Accuracy')
    plt.xlabel('Batch #')

    plt.show()
    fig_path = Path(outputs_dir) / 'test.jpg'
    fig.savefig(fig_path)


def plot_color_gradients(cmap_list, labels=None, title=''):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows-1)*0.1)*0.22
    fig, axs = plt.subplots(nrows=nrows, figsize=(6.4, figh))
    fig.subplots_adjust(top=1-.35/figh, bottom=.15/figh, left=0.2, right=0.99)

    axs[0].set_title(title, fontsize=14)
    gradient = np.linspace(0, 1, 256)
    gradient = np.vstack((gradient, gradient))

    i=0
    for ax, cmap_name in zip(axs, cmap_list):
        if labels == None:
            ax.imshow(gradient, aspect='auto', cmap=cmap_name)
            ax.text(-.01, .5, f'{i}', va='center', ha='right', fontsize=10,
                transform=ax.transAxes)
        else:
            if i not in labels:
                break
            ax.imshow(gradient, aspect='auto', cmap=cmap_name)
            ax.text(-.01, .5, f'{labels[i]}', va='center', ha='right', fontsize=10,
                transform=ax.transAxes)
        i+=1

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()
        
    plt.show()
    fig_path = Path(attention_matrices_dir) / f'{title}.jpg'
    fig.savefig(fig_path, bbox_inches='tight', pad_inches=0)


In [ ]:
# bins = pd.cut(mat.ravel(), bins=20)
#     shannon_entropy=0
#     tot_count = len(mat.ravel())
#     pjs = pd.value_counts(bins) / tot_countdicta = {'a': [1,2,3,4], 'b': [1,3,5,7], "c": [5,12,13]}
# lista = []
# for k, v in dicta.items():
#     lista.extend(v)
# unique, count = np.unique(lista, return_counts= True)
# list1occ = [x for idx, x in enumerate(unique) if count[idx]==1]
# dictaf = {k:[x for x in v if x in list1occ] for k,v in dicta.items()}
 

#### Test

In [ ]:
if do_test:
    if config_dict['TASK_TYPE']=='eigenvalues_analysis' or config_dict['TASK_TYPE']=='attention_analysis' or config_dict['TASK_TYPE']=='singularvalues_ratio_analysis' or config_dict['TASK_TYPE']=='distance_cones_analysis' or config_dict['TASK_TYPE']=='layer_output_analysis':
        selected_layer_head_list = config_dict['SELECTED_LAYER_HEAD_LIST']
        
    with open(log_file, 'a') as log_fp:
        attention_cases = []
        input_embs_identity = torch.ones(config_dict['MAX_LENGTH'] , config_dict['EMB_DIM'])
        input_embs_identity = input_embs_identity[None, :]  # add one dummy dimension, representing batch
        print("\n------- Identity embeddings, base model -------")
        _, attentions_id_base = supervisedBertClassifierTest(
            input_embs_identity,
            model_base, 
            log_fp, 
            config_dict['THETA'], 
            selected_layer_head_list=selected_layer_head_list, 
            dir_path=math_interpret_dir,
            title="identity_embeddings_base_model"
            )
        attention_cases.append({
            'title': "identity_embeddings_base_model",
            'attentions_dict': attentions_id_base
            })
        
        print("\n------- Identity embeddings, fine-tuned model -------")
        _, attentions_id_finetuned = supervisedBertClassifierTest(
            input_embs_identity,
            model_finetuned, 
            log_fp, 
            config_dict['THETA'], 
            selected_layer_head_list=selected_layer_head_list, 
            dir_path=math_interpret_dir,
            title="identity_embeddings_finetuned_model"
            )
        attention_cases.append({
            'title': "identity_embeddings_finetuned_model",
            'attentions_dict': attentions_id_finetuned
            })
        
        input_embs_randn = torch.randn(config_dict['MAX_LENGTH'] , config_dict['EMB_DIM'])
        input_embs_randn = input_embs_randn[None, :]  # add one dummy dimension, representing batch
        print("\n------- White noise embeddings, base model -------")
        _, attentions_rand_base = supervisedBertClassifierTest(
            input_embs_randn,
            model_base, 
            log_fp, 
            config_dict['THETA'], 
            selected_layer_head_list=selected_layer_head_list, 
            dir_path=math_interpret_dir,
            title="whitenoise_embeddings_base_model"
            )
        attention_cases.append({
            'title': "whitenoise_embeddings_base_model",
            'attentions_dict': attentions_rand_base
            })
        
        print("\n------- White noise embeddings, fine-tuned model -------")
        _, attentions_rand_finetuned = supervisedBertClassifierTest(
            input_embs_randn,
            model_finetuned, 
            log_fp, 
            config_dict['THETA'], 
            selected_layer_head_list=selected_layer_head_list, 
            dir_path=math_interpret_dir,
            title="whitenoise_embeddings_finetuned_model"
            )
        attention_cases.append({
            'title': "whitenoise_embeddings_finetuned_model",
            'attentions_dict': attentions_rand_finetuned
            })


###Attention analysis

In [ ]:
def get_attentions_for_case(attention_case):    
    if config_dict['TASK_TYPE']=='attention_analysis' or config_dict['TASK_TYPE']=='attention_flow' or config_dict['TASK_TYPE']=='von_neumann_entropy_attentions' or config_dict['TASK_TYPE']=='shannon_entropy_attentions':

        attention_matrices_file = Path(attention_matrices_dir) / f"{attention_case['title']}_attention_matrices_np"
        attention_matrices_thresh_file = Path(attention_matrices_dir) / f"{attention_case['title']}_attention_matrices_thresh_np"
        ticks_file = Path(attention_matrices_dir) / f"{attention_case['title']}_ticks_np"
        attentions2 = {}

        if os.path.exists(attention_matrices_file):
            attentions2['attentions_all_layers'] = pickle.load(open(attention_matrices_file, 'rb'))
        else: 
            pickle.dump(attention_case['attentions_dict']['attentions_all_layers'], open(attention_matrices_file, 'wb'))
            attentions2['attentions_all_layers'] = attention_case['attentions_dict']['attentions_all_layers']

        if os.path.exists(attention_matrices_thresh_file):
            attentions2['attentions_all_layers_thresh'] = pickle.load(open(attention_matrices_thresh_file, 'rb'))
        else: 
            pickle.dump(attention_case['attentions_dict']['attentions_all_layers_thresh'], open(attention_matrices_thresh_file, 'wb'))
            attentions2['attentions_all_layers_thresh'] = attention_case['attentions_dict']['attentions_all_layers_thresh']

        if os.path.exists(ticks_file):
            attentions2['repr_token_base_positions_axis'] = pickle.load(open(ticks_file, 'rb'))
        else: 
            pickle.dump(attention_case['attentions_dict']['repr_token_base_positions_axis'], open(ticks_file, 'wb'))
            attentions2['repr_token_base_positions_axis'] = attention_case['attentions_dict']['repr_token_base_positions_axis']
        
        return attentions2

In [ ]:
# np.asarray(attentions2['attentions_all_layers'][7][9][11]).max()

In [ ]:
def attention_analysis(attentions, log_fp_test, theta=0, selected_layers=None, selected_heads=None, title=None):
    # token_base_positions_axis = attentions['token_base_positions_axis']
    # attentions_last_layer = attentions['attentions_last_layer']
    attentions_all_layers = attentions['attentions_all_layers'] if 'attentions_all_layers' in attentions.keys() else None
    attentions_all_layers_thresh = attentions['attentions_all_layers_thresh']
    repr_token_base_positions_axis = attentions['repr_token_base_positions_axis']
    
    attn_last_layers_sum_dir = Path(attention_matrices_dir) / "last_layers_sum"

    for layer in selected_layers:
        if selected_heads == 'avg':
            #calculate the avg of attention matrices of heads of current layer
            attn_mat = attentions_all_layers[layer].mean(dim=0).cpu().detach().numpy() 
            plt_attentions(attn_mat, 
                        repr_token_base_positions_axis, 
                        attn_last_layers_sum_dir, 
                        theta=theta,
                        filename=f"{title}_{layer+1}",
                        title=f"Average of attention matrices of heads of layer {layer+1} for the case '{title}', theta = {theta:.3f}",
                        # cmap=colormaps_layers[layer]
                        ) 
        else:
            for head in selected_heads:
                attn_mat = attentions_all_layers[layer][head].cpu().detach().numpy() 
                plt_attentions(attn_mat, 
                        repr_token_base_positions_axis, 
                        attn_last_layers_sum_dir, 
                        theta=theta,
                        filename=f"{title}_{layer+1}_{head+1}",
                        title=f"Attention matrix of head {head+1} of layer {layer+1}  for the case '{title}', theta = {theta:.3f}",
                        # cmap=colormaps_layers[layer]
                        ) 

def attention_analysis_proportions(attentions, log_fp_test, theta=0):
    attentions_all_layers = attentions['attentions_all_layers'] if 'attentions_all_layers' in attentions.keys() else None
    # attentions_all_layers_thresh = attentions['attentions_all_layers_thresh']
    ##### ATTENTIONS BY DOMAIN ######
    show_attentions_for_each_domain(attentions_all_layers, proportion_attn_domains_dir, proportion="attention")
    # show_attentions_for_each_domain(attentions_all_layers_thresh, proportion_attn_domains_dir, proportion="high-attention-tokens-count", theta=theta)
    # attention_pct, pvalues = top_heads_attention_proportions(attentions_all_layers_thresh, proportion="high-confidence-attention", min_total=0, theta=theta)
    # print(f"\nPercentages of attention of maximally attentive heads:\n")
    # print(attention_pct.to_markdown())
    # print(f"\nP-values of attention of maximally attentive heads with respect to background frequencies:\n")
    # print(pvalues.to_markdown())
    # log_fp_test.write(f"\nP-values of attention of maximally attentive heads with respect to background frequencies:\n")
    # log_fp_test.write(f"{pvalues.to_markdown()}\n")
    # log_fp_test.write(f"\nPercentages of attention of maximally attentive heads:\n")
    # log_fp_test.write(f"{attention_pct.to_markdown()}\n")


In [ ]:
if config_dict['TASK_TYPE']=='attention_analysis':
    with open(log_file, 'a') as log_fp:
        for attention_case in attention_cases:
            attentions2 = get_attentions_for_case(attention_case)
            print(attentions2['repr_token_base_positions_axis'])
            selected_layers = range(n_layers) #[10] #range(n_layers)
            selected_heads = range(n_heads) #[4] #range(n_heads) #'avg'
            theta_plot = 0.01 #0.01
            attention_analysis(attentions2, log_fp, theta_plot, selected_layers, selected_heads, attention_case['title'])
            #attention_analysis_proportions(attentions2, log_fp) #theta=0.3

### Attention flow


In [ ]:
#@title Utilities

def get_adjmat(mat, input_tokens):
    n_layers, length, _ = mat.shape
    adj_mat = np.zeros(((n_layers+1)*length, (n_layers+1)*length))
    labels_to_index = {}
    for k in np.arange(length):
        labels_to_index[str(k)+"_"+input_tokens[k]] = k

    for i in np.arange(1,n_layers+1):
        for k_f in np.arange(length):
            index_from = (i)*length+k_f
            label = "L"+str(i)+"_"+str(k_f)
            labels_to_index[label] = index_from
            for k_t in np.arange(length):
                index_to = (i-1)*length+k_t
                adj_mat[index_from][index_to] = mat[i-1][k_f][k_t]
                
    return adj_mat, labels_to_index 


def draw_attention_graph(adjmat, labels_to_index, n_layers, length):
    A = adjmat
    G=nx.from_numpy_matrix(A, create_using=nx.DiGraph())
    for i in np.arange(A.shape[0]):
        for j in np.arange(A.shape[1]):
            nx.set_edge_attributes(G, {(i,j): A[i,j]}, 'capacity')

    pos = {}
    label_pos = {}
    for i in np.arange(n_layers+1):
        for k_f in np.arange(length):
            pos[i*length+k_f] = ((i+0.5)*2, length - k_f)
            label_pos[i*length+k_f] = (i*2, length - k_f)

    index_to_labels = {}
    for key in labels_to_index:
        index_to_labels[labels_to_index[key]] = key.split("_")[-1]
        if labels_to_index[key] >= length:
            index_to_labels[labels_to_index[key]] = ''

    #plt.figure(1,figsize=(20,12))

    nx.draw_networkx_nodes(G,pos,node_color='green', node_size=50)
    nx.draw_networkx_labels(G,pos=label_pos, labels=index_to_labels, font_size=10)

    all_weights = []
    #4 a. Iterate through the graph nodes to gather all the weights
    for (node1,node2,data) in G.edges(data=True):
        all_weights.append(data['weight']) #we'll use this when determining edge thickness

    #4 b. Get unique weights
    unique_weights = list(set(all_weights))

    #4 c. Plot the edges - one by one!
    print("Plot the edges - one by one!")
    for weight in tqdm(unique_weights):
        #4 d. Form a filtered list with just the weight you want to draw
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in G.edges(data=True) if edge_attr['weight']==weight]
        #4 e. I think multiplying by [num_nodes/sum(all_weights)] makes the graphs edges look cleaner
        
        w = weight #(weight - min(all_weights))/(max(all_weights) - min(all_weights))
        width = w
        nx.draw_networkx_edges(G,pos,edgelist=weighted_edges,width=width, edge_color='darkblue')
    
    return G

def get_attention_graph(adjmat, labels_to_index, n_layers, length):
    A = adjmat
    G=nx.from_numpy_matrix(A, create_using=nx.DiGraph())
    for i in np.arange(A.shape[0]):
        for j in np.arange(A.shape[1]):
            nx.set_edge_attributes(G, {(i,j): A[i,j]}, 'capacity')

    pos = {}
    label_pos = {}
    for i in np.arange(n_layers+1):
        for k_f in np.arange(length):
            pos[i*length+k_f] = ((i+0.5)*2, length - k_f)
            label_pos[i*length+k_f] = (i*2, length - k_f)

    index_to_labels = {}
    for key in labels_to_index:
        index_to_labels[labels_to_index[key]] = key.split("_")[-1]
        if labels_to_index[key] >= length:
            index_to_labels[labels_to_index[key]] = ''

    #plt.figure(1,figsize=(20,12))

    nx.draw_networkx_nodes(G,pos,node_color='green', node_size=50)
    nx.draw_networkx_labels(G,pos=label_pos, labels=index_to_labels, font_size=10)

    all_weights = []
    #4 a. Iterate through the graph nodes to gather all the weights
    for (node1,node2,data) in G.edges(data=True):
        all_weights.append(data['weight']) #we'll use this when determining edge thickness

    #4 b. Get unique weights
    unique_weights = list(set(all_weights))

    #4 c. Plot the edges - one by one!
    for weight in unique_weights:
        #4 d. Form a filtered list with just the weight you want to draw
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in G.edges(data=True) if edge_attr['weight']==weight]
        #4 e. I think multiplying by [num_nodes/sum(all_weights)] makes the graphs edges look cleaner
        
        w = weight #(weight - min(all_weights))/(max(all_weights) - min(all_weights))
        width = w
        nx.draw_networkx_edges(G,pos,edgelist=weighted_edges,width=width, edge_color='darkblue')
    
    return G

def compute_flows(G, labels_to_index, input_nodes, length):
    print("Computing flows")
    number_of_nodes = len(labels_to_index)
    flow_values=np.zeros((number_of_nodes,number_of_nodes))
    for key in tqdm(labels_to_index):
        if key not in input_nodes:
            current_layer = int(labels_to_index[key] / length)
            pre_layer = current_layer - 1
            u = labels_to_index[key]
            for inp_node_key in input_nodes:
                v = labels_to_index[inp_node_key]
                flow_value = nx.maximum_flow_value(G,u,v, flow_func=nx.algorithms.flow.edmonds_karp)
                flow_values[u][pre_layer*length+v ] = flow_value
            flow_values[u] /= flow_values[u].sum()
            
    return flow_values

def compute_node_flow(G, labels_to_index, input_nodes, output_nodes,length):
    number_of_nodes = len(labels_to_index)
    flow_values=np.zeros((number_of_nodes,number_of_nodes))
    for key in output_nodes:
        if key not in input_nodes:
            current_layer = int(labels_to_index[key] / length)
            pre_layer = current_layer - 1
            u = labels_to_index[key]
            for inp_node_key in input_nodes:
                v = labels_to_index[inp_node_key]
                flow_value = nx.maximum_flow_value(G,u,v, flow_func=nx.algorithms.flow.edmonds_karp)
                flow_values[u][pre_layer*length+v ] = flow_value
            flow_values[u] /= flow_values[u].sum()
            
    return flow_values

def compute_joint_attention(att_mat, add_residual=True):
    if add_residual:
        residual_att = np.eye(att_mat.shape[1])[None,...]
        aug_att_mat = att_mat + residual_att
        aug_att_mat = aug_att_mat / aug_att_mat.sum(axis=-1)[...,None]
    else:
       aug_att_mat =  att_mat
    
    joint_attentions = np.zeros(aug_att_mat.shape)

    layers = joint_attentions.shape[0]
    joint_attentions[0] = aug_att_mat[0].numpy()
    for i in np.arange(1,layers):
        joint_attentions[i] = aug_att_mat[i].numpy().dot(joint_attentions[i-1])
        
    return joint_attentions

def plot_attention_heatmap(att, s_position, t_positions, sentence):

  cls_att = np.flip(att[:,s_position, t_positions], axis=0)
  xticklb = input_tokens= list(itertools.compress(['<cls>']+sentence.split(), [i in t_positions for i in np.arange(len(sentence)+1)]))
  yticklb = [str(i) if i%2 ==0 else '' for i in np.arange(att.shape[0],0, -1)]
  ax = sns.heatmap(cls_att, xticklabels=xticklb, yticklabels=yticklb, cmap="YlOrRd")
  return ax


def convert_adjmat_tomats(adjmat, n_layers, l):
   mats = np.zeros((n_layers,l,l))
   
   for i in np.arange(n_layers):
       mats[i] = adjmat[(i+1)*l:(i+2)*l,i*l:(i+1)*l]
       
   return mats

def compute_raw_attention_residual_connections(attentions_mat, input_tokens):
    print("Get raw attention mat + residual coonections")
    res_att_mat = attentions_mat.sum(axis=1)/attentions_mat.shape[1]
    res_att_mat = res_att_mat + np.eye(res_att_mat.shape[1])[None,...]
    res_att_mat = res_att_mat / res_att_mat.sum(axis=-1)[...,None]
    
    res_adj_mat, res_labels_to_index = get_adjmat(mat=res_att_mat, input_tokens=input_tokens)
    return res_adj_mat, res_labels_to_index, res_att_mat


In [ ]:
#title Get raw attention mat + residual coonections
def plot_raw_attention_residual_connections(attentions_mat, input_tokens):
    print("\n============ RAW ATTENTION MAP + RESIDUAL CONNECTIONS ============")
    res_adj_mat, res_labels_to_index, res_att_mat = compute_raw_attention_residual_connections(attentions_mat=attentions_mat, input_tokens=input_tokens)

    plt.figure(figsize=(20,100))
    res_G = draw_attention_graph(res_adj_mat,res_labels_to_index, n_layers=res_att_mat.shape[0], length=res_att_mat.shape[-1])
    return res_G

def plot_attention_rollout(attentions_mat, input_tokens):
    print("\n============ ATTENTION ROLLOUT ============")
    res_adj_mat, res_labels_to_index, res_att_mat = compute_raw_attention_residual_connections(attentions_mat=attentions_mat, input_tokens=input_tokens)
    joint_attentions = compute_joint_attention(res_att_mat, add_residual=False)
    joint_att_adjmat, joint_labels_to_index = get_adjmat(mat=joint_attentions, input_tokens=input_tokens)
    plt.figure(figsize=(20,100))
    G = draw_attention_graph(joint_att_adjmat,joint_labels_to_index, n_layers=joint_attentions.shape[0], length=joint_attentions.shape[-1])

def plot_attention_flow(attentions_mat, input_tokens, res_G):
    print("\n============ ATTENTION FLOW ============")
    res_adj_mat, res_labels_to_index, res_att_mat = compute_raw_attention_residual_connections(attentions_mat=attentions_mat, input_tokens=input_tokens)
    print("Compute attention flow (this will take quite some time to compute)")
    output_nodes = []
    input_nodes = []
    for key in tqdm(res_labels_to_index):
        if 'L24' in key:
            output_nodes.append(key)
        if res_labels_to_index[key] < attentions_mat.shape[-1]:
            input_nodes.append(key)

    flow_values = compute_flows(res_G, res_labels_to_index, input_nodes, length=attentions_mat.shape[-1])

    plt.figure(figsize=(20,100))
    flow_G = draw_attention_graph(flow_values,res_labels_to_index, n_layers=attentions_mat.shape[0], length=attentions_mat.shape[-1])


def attention_flow(attentions, log_fp_test, selected_classes=None, selected_layers=None, selected_heads=None):
    # token_base_positions_axis = attentions['token_base_positions_axis']
    # attentions_last_layer = attentions['attentions_last_layer']

    for target_label in selected_classes:
        attentions_mat = attentions['attentions_all_layers'][target_label]
        tokens = attentions['repr_token_base_positions_axis'][target_label]
        # domain_start_idx, domain_end_idx = convert_domain_coords_in_token_indices("NTD")
        domain_end_idx = 12
        print(domain_end_idx)
        attentions_mat = attentions_mat[:,:,:domain_end_idx,:domain_end_idx]
        tokens = tokens[:domain_end_idx] 
        print(attentions_mat.shape)
        res_G = plot_raw_attention_residual_connections(attentions_mat, tokens)
        plot_attention_rollout(attentions_mat, tokens)
        plot_attention_flow(attentions_mat, tokens, res_G)



In [ ]:
#@title Analysis
if config_dict['TASK_TYPE']=='attention_flow':
    with open(log_file, 'a') as log_fp:
            selected_classes = [3] #[1]  #range(config_dict['N_CLASSES'])
            attention_flow(attentions2, log_fp, selected_classes)


###Entropy of attention matrices

In [ ]:
def entropy_attentions(type_entropy, attentions, selected_classes, selected_layers, selected_heads, file_fp):
    attentions_all_layers = attentions['attentions_all_layers']
    
    for target_label in selected_classes:
        file_fp.write(f"{type_entropy} entropy {'of eigenvalues ' if type_entropy=='Von-Neumann' else ''}of attention matrices, class '{inv_class_labels_dict[int(target_label)]}'\n")            
    
        VN_entropy_layer_attn_sum = np.zeros(n_layers)
        Sh_entropy_layer_attn_sum = np.zeros(n_layers)

        for layer in selected_layers:
            
            if selected_heads == 'avg':
                #calculate the avg of attention matrices of heads of current layer
                attn_mat_layer = attentions_all_layers[target_label][layer].mean(dim=0).cpu().detach().numpy() 
                if type_entropy=="von_neumann_entropy_attentions":
                    result = compute_Von_Neumann_entropy_eigvals(attn_mat_layer, f"layer_{layer+1}")
                elif type_entropy=="shannon_entropy_attentions":
                    result = compute_Shannon_entropy(attn_mat_layer, f"layer_{layer+1}")
                print(f"\t{type_entropy}_layer_{layer+1} = {result}")
                file_fp.write(f"\t{type_entropy}_layer_{layer+1} = {result}\n")

            else:
                for head in selected_heads:
                    attn_mat_head = attentions_all_layers[target_label][layer][head].cpu().detach().numpy() 
                    if type_entropy=="von_neumann_entropy_attentions":
                        result = compute_Von_Neumann_entropy_eigvals(attn_mat_head, f"head_{head+1}_layer_{layer+1}")
                    elif type_entropy=="shannon_entropy_attentions":
                        result = compute_Shannon_entropy(attn_mat_head, f"head_{head+1}_layer_{layer+1}")
                    print(f"\t{type_entropy}_entropy_head_{head+1}_layer_{layer+1} = {result}")
                    file_fp.write(f"\t{type_entropy}_entropy_head_{head+1}_layer_{layer+1} = {result}\n")

def plot_entropies(attentions, selected_classes, selected_layers, selected_heads):
    attentions_all_layers = attentions['attentions_all_layers']
    VN_entropy_layer_attn_sum = np.zeros(n_layers)
    Sh_entropy_layer_attn_sum = np.zeros(n_layers)
    
    for target_label in selected_classes:
        for layer in selected_layers:
            for head in selected_heads:

                attn_mat_head = attentions_all_layers[target_label][layer][head].cpu().detach().numpy() 

                VN_entropy_attn = compute_Von_Neumann_entropy_eigvals(attn_mat_head, f"head{head+1}_layer_{layer+1}")
                Sh_entropy_attn = compute_Shannon_entropy(attn_mat_head, f"head{head+1}_layer_{layer+1}")

                VN_entropy_layer_attn_sum[layer] += VN_entropy_attn
                Sh_entropy_layer_attn_sum[layer] += Sh_entropy_attn
            
        VN_entropy_layer_attn_mean = VN_entropy_layer_attn_sum / len(selected_heads)
        Sh_entropy_layer_attn_mean = Sh_entropy_layer_attn_sum / len(selected_heads)


        # x = [i+1 for i in range(n_layers)]
        fig, ax = plt.subplots(figsize=(6,6))
        ax.plot(Sh_entropy_layer_attn_mean, 'b', label='Shannon entropy')
        # ax.axis('equal')
        leg = ax.legend()
        plt.title(f"Average Shannon entropy of attention matrices of each layer")
        plt.grid(linestyle = '--')
        ax.set_xticks(range(n_layers))
        ax.set_xticklabels(np.asarray(ax.get_xticks())+1)
        #fig.yticks(list(plt.yticks()[0]) + [1])
        ax.set_xlabel("Layer")
        ax.set_ylabel("Shannon entropy")
        plt.show()
        fig_path = Path(math_interpret_dir) / f'Shannon_entropy_plot.jpg'
        fig.savefig(fig_path, bbox_inches='tight')
        plt.close()


        # x = [i+1 for i in range(n_layers)]
        fig, ax = plt.subplots(figsize=(6,6))
        ax.plot(VN_entropy_layer_attn_mean, 'b', label='VN entropy')
        # ax.axis('equal')
        leg = ax.legend()
        plt.title(f"Average Von Neumann entropy of attention matrices of each layer")
        plt.grid(linestyle = '--')
        ax.set_xticks(range(n_layers))
        ax.set_xticklabels(np.asarray(ax.get_xticks())+1)
        #fig.yticks(list(plt.yticks()[0]) + [1])
        ax.set_xlabel("Layer")
        ax.set_ylabel("VN entropy")
        plt.show()
        fig_path = Path(math_interpret_dir) / f'VN_entropy_plot.jpg'
        fig.savefig(fig_path, bbox_inches='tight')
        plt.close()
                        

In [ ]:
if config_dict['TASK_TYPE']=='von_neumann_entropy_attentions' or config_dict['TASK_TYPE']=='shannon_entropy_attentions':
    attn_eigvals_file = Path(math_interpret_dir) / f"{config_dict['TASK_TYPE']}.txt"
    with open(attn_eigvals_file, 'w') as attn_eigvals_fp:

        selected_classes = [7]  #attentions_all_layers.keys()
        selected_layers = range(n_layers)
        # selected_heads = 'avg'
        # entropy_attentions(config_dict['TASK_TYPE'], attentions2, selected_classes, selected_layers, selected_heads, attn_eigvals_fp)
        selected_heads = range(n_heads)
        # entropy_attentions(config_dict['TASK_TYPE'], attentions2, selected_classes, selected_layers, selected_heads, attn_eigvals_fp)

    plot_entropies(attentions2, selected_classes, selected_layers, selected_heads)

### Analysis of eigenvalues of weights of Qi and Ki

In [ ]:
# model.parameters
print(f'{"parameter": <55}: {"mean": <20} (std)')
print()
for name,param in model.named_parameters():
    if name.endswith("attention.output.LayerNorm.weight"):
        print(f'{name: <55}: {torch.mean(param): <20} ({torch.std(param)})')
print()
for name,param in model.named_parameters():
    if name.endswith("attention.output.LayerNorm.bias"):
        print(f'{name: <55}: {torch.mean(param): <20} ({torch.std(param)})')
print()
for name,param in model.named_parameters():
    if name.endswith("output.LayerNorm.weight") and not name.endswith("attention.output.LayerNorm.weight"):
        print(f'{name: <55}: {torch.mean(param): <20} ({torch.std(param)})')
print()
for name,param in model.named_parameters():
    if name.endswith("output.LayerNorm.bias") and not name.endswith("attention.output.LayerNorm.bias"):
        print(f'{name: <55}: {torch.mean(param): <20} ({torch.std(param)})')
# print(model.parameters)

# model_params = dict(model.named_parameters())
# model_params["encoder.layer.0.attention.output.LayerNorm.bias"].data.cpu().detach().numpy()

In [ ]:
def compute_head_weights_eigenvalues(model, dir_path):
    layer_query_weight_names = [f'bert.encoder.layer.{l}.attention.self.query.weight' for l in range(n_layers)]
    layer_key_weight_names = [f'bert.encoder.layer.{l}.attention.self.key.weight' for l in range(n_layers)]
    model_params = dict(model.named_parameters())
    for layer in np.flip(range(n_layers)):
        w_q = model_params[layer_query_weight_names[layer]].data
        w_k = model_params[layer_key_weight_names[layer]].data
        emb_dim = w_q.shape[0]
        d_k = int(emb_dim / n_heads)
                
        weights_prod = np.matmul(w_q, w_k.T)

        eigenvals = np.linalg.eigvals(weights_prod)
        print(eigenvals)
        return
        eigenvals_count = {}
        eigenvals_count['positive'] = sum([1 for x in eigenvals if x>0])
        eigenvals_count['negative'] = sum([1 for x in eigenvals if x<0])
        eigenvals_count['zero'] = sum([1 for x in eigenvals if x==0])
        
        #eigenvals_count = collections.Counter(eigenvals)
        print(f"n. unique eigenvalues = {len(np.unique(eigenvals))}")
        fig=plt.figure(figsize=(8,8))
        # print(np.sort_complex(eigenvals)[:10])
        plt.bar(range(len(eigenvals_count.keys())), eigenvals_count.values(), align='center', width=0.8)
        plt.xticks(range(len(eigenvals_count.keys())), eigenvals_count.keys(), rotation=45, fontsize=12)
        plt.title(f"Eigenvalues for layer {layer+1}")
        plt.show()
        fig_path = Path(dir_path) / f'{layer+1}.jpg'
        fig.savefig(fig_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        # for i in range(n_heads):
        #     w_q_i = w_q[:, i*d_k : i*d_k + d_k]
        #     w_k_i = w_k[:, i*d_k : i*d_k + d_k]
        #     weights_prod = np.matmul(w_q_i, w_k_i.T)
        #     eigenvals = np.linalg.eigvals(weights_prod)
        #     eigenvals_count = {}
        #     eigenvals_count['positive'] = sum([1 for x in eigenvals if x>0])
        #     eigenvals_count['negative'] = sum([1 for x in eigenvals if x<0])
        #     eigenvals_count['zero'] = sum([1 for x in eigenvals if x==0])
            
        #     #eigenvals_count = collections.Counter(eigenvals)
        #     print(f"n. unique eigenvalues = {len(np.unique(eigenvals))}")
        #     plt.figure(figsize=(8,8))
        #     # print(np.sort_complex(eigenvals)[:10])
        #     plt.bar(range(len(eigenvals_count.keys())), eigenvals_count.values(), align='center', width=0.8)
        #     plt.xticks(range(len(eigenvals_count.keys())), eigenvals_count.keys(), rotation=45, fontsize=12)
        #     plt.title(f"Eigenvalues for head {i+1} layer {layer+1}")
        #     plt.show()
        #     fig_path = Path(dir_path) / f'{layer+1}_{i+1}.jpg'
        #     fig.savefig(fig_path, bbox_inches='tight', pad_inches=0)
        #     plt.close()

def compute_max_bias(model):
    layer_query_weight_names = [f'bert.encoder.layer.{l}.attention.self.query.bias' for l in range(n_layers)]
    layer_key_weight_names = [f'bert.encoder.layer.{l}.attention.self.key.bias' for l in range(n_layers)]
    model_params = dict(model.named_parameters())
    for layer in np.flip(range(n_layers)):
            bias_q = model_params[layer_query_weight_names[layer]].data
            bias_k = model_params[layer_key_weight_names[layer]].data
            print(f"layer {layer}")
            print(f"\tbias_q={len(bias_q)}")
            print(f"\tbias_k={len(bias_k)}")

def hist_Wq_WkT(model):
    layer_query_weight_names = [f'bert.encoder.layer.{l}.attention.self.query.weight' for l in range(n_layers)]
    layer_key_weight_names = [f'bert.encoder.layer.{l}.attention.self.key.weight' for l in range(n_layers)]
    model_params = dict(model.named_parameters())
    
    for layer in np.flip(range(n_layers)):
        W_q = model_params[layer_query_weight_names[layer]].data.cpu().detach().numpy()
        W_k = model_params[layer_key_weight_names[layer]].data.cpu().detach().numpy()
        emb_dim = W_q.shape[0]
        d_k = int(emb_dim / n_heads)
        head = 0
        W_q_i = W_q[:, head*d_k : head*d_k + d_k]
        W_k_i = W_k[:, head*d_k : head*d_k + d_k]        
        weights_prod = np.matmul(W_q_i, W_k_i.T)
        fig, ax = plt.subplots(figsize=(15,10))
        ax.set_aspect('equal')
        sns.heatmap(weights_prod)
        plt.show()
        
        heat_dir = Path(math_interpret_dir) / "heatmaps_Wq_WkT"
        if not os.path.exists(heat_dir):
            os.makedirs(heat_dir)
            print(f"Directory '{heat_dir}' created")
        fig_path = Path(heat_dir) / f'heatmap_Wq_WkT_L{layer+1}_H1.jpg'
        fig.savefig(fig_path)


In [ ]:
if config_dict['TASK_TYPE'] == 'eigenvalues_analysis':
    head_weights_eigenvalues = compute_head_weights_eigenvalues(model, math_interpret_dir)
    compute_max_bias(model)
    hist_Wq_WkT(model)